# 🥵 Feversum Dataset 2nd take

In [1]:
import os, pandas as pd

# list all tsv, json and jsonl files in subfolders of PATH and print their names
key = "clef2022_task1_rc_twitter"
PATH = "/mnt/data/factcheck/claim_extraction"
paths = []


def load_df(path):
    if path.endswith(".tsv"):
        df = pd.read_csv(path, sep="\t")
    elif path.endswith(".csv"):
        df = pd.read_csv(path)
    elif path.endswith(".json"):
        df = pd.read_json(path)
    elif path.endswith(".jsonl"):
        df = pd.read_json(path, lines=True)
    return df



In [10]:
test_df = load_df("/mnt/data/factcheck/fever/2022_error_correction/intermediate_masks/train_dev_gold.jsonl")

In [12]:
# pd char limit to 64
pd.options.display.max_colwidth = 64
# from [(a,b), (c,d),..] couples in pipeline_text only persist the first element
test_df["pipeline_sents"] = test_df["pipeline_text"].apply(lambda x: [y[1] for y in x] )
test_df["pipeline_text"] = test_df["pipeline_text"].apply(lambda x: [y[0] for y in x] )
test_df.sample(3)

,sentence1,sentence2,sentence1_masked,masked_inds,target,num_masked,gold_label,label_prob,succeed,original,...,original_claim,mutation,pipeline_text,claim_id,mutated,original_id,sentence_id,evidence,metadata,pipeline_sents
66711,Hunter S. Thompson did not die at the age of 67 .,"After a bout of health problems , Thompson committed suicide...",Hunter S. Thompson did not die at the $$ of $$ $$,"[8, 10, 11]",Hunter S. Thompson died at age 67.,3,REFUTES,0.999583,1,{'source': 'Hunter S. Thompson did not die at the age of 67....,...,Hunter S. Thompson did not die at the age of 67 .,negate,"[Hunter S. Thompson, Hunter S. Thompson]",154999,NaN,NaN,NaN,NaN,NaN,"[: `` I hate to advocate drugs , alcohol , violence , or ins..."
37319,London is the location of a place .,The City of London is a city and county that is an enclave o...,London $$ the $$ of a place $$,"[1, 3, 7]",London is the location of an enclave.,3,SUPPORTS,0.927977,1,"{'source': 'London is the location of a place.', 'target': '...",...,London is the location of a place .,general,"[London, London]",82778,NaN,NaN,NaN,NaN,NaN,"[marks the Prime Meridian , 0 ° longitude , and GMT -RRB- . ..."
65147,Bryan Cranston acted in Breaking Bad .,It tells the story of Walter White -LRB- Bryan Cranston -RRB...,Bryan Cranston acted in $$ Bad $$,"[4, 6]",Bryan Cranston won an Emmy 4 times for his role in Breaking ...,2,SUPPORTS,0.941516,1,"{'source': 'Bryan Cranston acted in Breaking Bad.', 'target'...",...,Bryan Cranston acted in Breaking Bad .,substitute_dissimilar,"[Bryan Cranston, Breaking Bad]",151052,NaN,NaN,NaN,NaN,NaN,"[Middle , and Dr. Tim Whatley in five episodes of the NBC co..."


In [7]:
# see "original" va

In [13]:
# add a column that contains original[origina][original] value
# iterate rows, see where original[original][original] is different from target
crashes = 0
for i, row in test_df.iterrows():
    try:
        
        original = row["original"]["original"]["original"]
        target = row["target"]
        if original != target:
            print(original)
            print(target)
            print("")
    except:
        crashes += 1
        
print(crashes, len(test_df))

6667 68271


In [14]:
# see where original column is not dict
test_df[test_df["original"].apply(lambda x: type(x) != dict)].sample(3).iloc[0]["metadata"]

{'source': 'Dopamine induces neuromodulation.',
 'target': 'Serotonin induces neuromodulation.',
 'evidence': 'Major neuromodulators in the central nervous system include dopamine , serotonin , acetylcholine , histamine , and norepinephrine .',
 'mutation_type': 'substitute_similar',
 'veracity': 'SUPPORTS'}

In [15]:
# toss columns where original is not dict
df = test_df[test_df["original"].apply(lambda x: type(x) == dict)]
# toss columns that dont define original[original][sentence_id]
df = df[df["original"].apply(lambda x: "sentence_id" in x["original"])]
# add column source_sentence with value original[original][sentence_id]
df["source_sentence"] = df["original"].apply(lambda x: x["original"]["sentence_id"])

In [16]:
df

,sentence1,sentence2,sentence1_masked,masked_inds,target,num_masked,gold_label,label_prob,succeed,original,...,mutation,pipeline_text,claim_id,mutated,original_id,sentence_id,evidence,metadata,pipeline_sents,source_sentence
0,Julianne Moore was not in the television series As the World...,"From 1985 to 1988 , she was a regular in the soap opera As t...",Julianne Moore was not in the $$ series As the $$ $$ $$,"[6, 10, 11, 12]",Julianne Moore was in the television series As the World Turns.,4,REFUTES,0.997579,1,{'source': 'Julianne Moore was not in the television series ...,...,negate,"[Julianne Moore, Julianne Moore filmography]",3305,NaN,NaN,NaN,NaN,NaN,[including the 2014 Academy Award for Best Actress . After s...,1455
1,Exercise is bad for heart health .,"Exercise temporarily increases the rate , but lowers resting...",Exercise $$ bad for $$ $$ $$,"[1, 4, 5, 6]",Exercise is good for heart health.,4,REFUTES,0.988043,1,"{'source': 'Exercise is bad for heart health.', 'target': 'E...",...,substitute_similar,"[Physical exercise, Physical exercise]",3518,NaN,NaN,NaN,NaN,NaN,[non-pharmaceutical sleep aid to treat diseases such as inso...,1542
2,A monster is always a person .,"A monster is any creature , usually found in legends or horr...",A monster $$ always a $$ $$,"[2, 5, 6]",A monster is a creature.,3,REFUTES,0.958360,1,"{'source': 'A monster is always a person.', 'target': 'A mon...",...,negate,"[Monster, Monster]",4172,NaN,NaN,NaN,NaN,NaN,"[, Frankenstein 's monster , werewolves , mummies , and zomb...",1781
3,Amyotrophic lateral sclerosis is a cure .,"Amyotrophic lateral sclerosis -LRB- ALS -RRB- , also known a...",Amyotrophic $$ $$ $$ a $$ $$,"[1, 2, 3, 5, 6]",Amyotrophic lateral sclerosis is a disease.,5,REFUTES,0.785549,1,"{'source': 'Amyotrophic lateral sclerosis is a cure.', 'targ...",...,negate,"[Amyotrophic lateral sclerosis, Amyotrophic lateral sclerosis]",5689,NaN,NaN,NaN,NaN,NaN,[rule out other potential causes . No cure for ALS is known ...,2287
4,Sandra Bullock worked in the film industry .,She was awarded the Academy Award for Best Actress and the G...,Sandra Bullock worked in the $$ $$ $$,"[5, 6, 7]",Sandra Bullock was an executive producer.,3,SUPPORTS,0.991370,1,"{'source': 'Sandra Bullock worked in the film industry.', 't...",...,general,"[Sandra Bullock, Sandra Bullock]",6757,NaN,NaN,NaN,NaN,NaN,"[office . In 2007 , she was one of Hollywood 's highest-paid...",2609
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68266,Joan of Arc is a French national symbol .,In the 16th century she became a symbol of the Catholic Leag...,Joan of Arc $$ a $$ $$ $$ $$,"[3, 5, 6, 7, 8]",Joan of Arc is a national symbol of France.,5,SUPPORTS,0.999116,1,"{'source': 'Joan of Arc is a French national symbol.', 'targ...",...,rephrase,"[Joan of Arc, Joan of Arc]",58087,NaN,NaN,NaN,NaN,NaN,[and in 1803 she was declared a national symbol of France by...,1964
68267,Wentworth premiered on ABC .,The show has received a mostly positive reception from criti...,Wentworth $$ on $$ $$,"[1, 3, 4]",Wentworth premiered on Foxtel.,3,REFUTES,0.996348,1,"{'source': 'Wentworth premiered on ABC.', 'target': 'Wentwor...",...,substitute_similar,"[Wentworth (season 1), Wentworth (TV series)]",58138,NaN,NaN,NaN,NaN,NaN,[The first season of the crime drama television series Wentw...,1555
68268,Clint Eastwood won at least two awards .,"In addition to directing many of his own star vehicles , Eas...",Clint Eastwood $$ at $$ two $$ $$,"[2, 4, 6, 7]",Clint Eastwood won at least two Academy Awards.,4,SUPPORTS,0.998482,1,"{'source': 'Clint Eastwood won at least two awards.', 'targe...",...,general,"[Clint Eastwood, Academy Award for Best Actor]",58163,NaN,NaN,NaN,NaN,NaN,"[critical praise in France for several films , including som...",4221
68269,AC / DC released zero albums .,"Later that year , the band released the new album , Back in ...",AC / DC released $$ $$ $$,"[4, 5, 6]","AC/DC released the album ""The Razor's E

In [7]:
from utils.ntbutils import *
load_user_libs()

In [8]:
import unicodedata as ud
NORM="NFC"
def normalize(key, replace_underscore=True):
    if not replace_underscore:
        return ud.normalize(NORM, detokenize2(key))
    return ud.normalize(NORM, detokenize2(key).replace("_"," "))

directory = "/mnt/data/factcheck/fever/data-en/wiki-pages"
wiki = {}
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        for datapoint in load_jsonl(f):
            wiki[normalize(datapoint["id"])] = normalize(datapoint["text"],False)


In [29]:
# show first row
df.sample(1).iloc[0]["original"]

{'source': 'Dwayne "The Rock" Johnson has never wrestled for WWE.',
 'target': 'Dwayne "The Rock" Johnson was a major figure in the Attitude Era of WWE.',
 'evidence': 'Johnson has 17 championship reigns in WWE , including 10 as a world champion , winning the WWF/WWE Championship eight times -LRB- seventh reign was as WWE Undisputed Champion -RRB- and the WCW/World Championship twice .',
 'mutation_type': 'negate',
 'veracity': 'REFUTES',
 'original': {'mutated': 'Dwayne "The Rock" Johnson has never wrestled for WWE.',
  'original': 'Dwayne "The Rock" Johnson was a major figure in the Attitude Era of WWE.',
  'mutation': 'negate',
  'claim_id': 41243,
  'original_id': 34609,
  'sentence_id': 3547,
  'verdict': 'REFUTES',
  'evidence': [{'annotation_id': 60291,
    'verdict_id': 70671,
    'page': 'Dwayne_Johnson',
    'line': 9},
   {'annotation_id': 60291,
    'verdict_id': 70672,
    'page': 'Dwayne_Johnson',
    'line': 10},
   {'annotation_id': 60291,
    'verdict_id': 70673,
    '

In [157]:
df_grp2 = df_grp.copy()

In [166]:
pd.set_option('display.max_colwidth', 512)
df_grp = df.groupby("source_sentence").agg({"target": set, "pipeline_text": "sum", "pipeline_sents": "sum"})
df_grp.rename(columns={"target": "claims"}, inplace=True)
df_grp["source"] = df_grp["pipeline_text"].apply(lambda x: max(set(x), key = x.count))
df_grp["source_text"]= df_grp["source"].apply(lambda x: wiki[normalize(x)])
df_grp["source_sent"] = df_grp["pipeline_sents"].apply(lambda x: max(set(x), key = x.count))
# normalize source_sent
df_grp["source_sent"] = df_grp["source_sent"].apply(normalize)
# flatten the df to neglect the pre-grouping data
df_grp = df_grp.reset_index()
df_grp = df_grp[["source_sentence", "claims", "source", "source_sent", "source_text", "pipeline_sents", "pipeline_text"]]
df_grp.head(4)

,source_sentence,claims,source,source_sent,source_text,pipeline_sents,pipeline_text
0,2,"{Chris Hemsworth appeared in A Perfect Getaway., Chris Hemsworth appeared in Star Trek., Chris Hemsworth is known for his role on the Australian TV series Home and Away., Chris Hemsworth is known for playing Thor in The Marvel Cinematic Universe., Chris Hemsworth is known for playing Kim Hyde.}",Chris Hemsworth,Chris Hemsworth (born 11 August 1983) is an Australian actor. He is known for playing Kim Hyde in the Australian TV series Home and Away (2004-07) and Thor in the Marvel Cinematic Universe since 2011. Hemsworth has also appeared in the science fiction action,"Chris Hemsworth (born 11 August 1983) is an Australian actor. He is known for playing Kim Hyde in the Australian TV series Home and Away (2004-07) and Thor in the Marvel Cinematic Universe since 2011. Hemsworth has also appeared in the science fiction action film Star Trek (2009), the thriller adventure A Perfect Getaway (2009), the horror comedy The Cabin in the Woods (2012), the dark-fantasy action film Snow White and the Huntsman (2012), the war film Red Dawn (2012), and the biographical sports drama...","[Kevin Michael Costner -LRB- born January 18 , 1955 -RRB- is an American actor , director , producer , and musician . His accolades include two Academy Awards , three Golden Globe Awards , and one Emmy Award . Costner has played Eliot Ness in The Untouchables , Crash Davis, Seth Gilliam -LRB- born November 5 , 1968 -RRB- is an American actor . He is best known for his portrayals of Ellis Carver on The Wire , Clayton Hughes on Oz , Dr. Alan Deaton on Teen Wolf , and Father Gabriel Stokes on The Walking D...","[Kevin Costner, Seth Gilliam, Shane Withington, Adam Willits, Chris Hemsworth, Chris Hemsworth, Thor (film), Rick Cosnett, T.J. Lowther, Donald Faison, Chris Jarvis (presenter), Chris Jarvis (presenter), Marshall Lancaster, Chris Hemsworth, Adam Willits, Shane Withington, Donald Faison, Chris Jarvis (presenter), Chris Hemsworth, Thor (film), Peter Cundall, Sam Kekovich, Chris Lilley (comedian), John Corbett, A Perfect Getaway, A Perfect Getaway, John Corbett, Donald Faison, Chris Hemsworth, A Perfect Ge..."
1,4,"{Roald Dahl is a writer., Roald Dahl was listed as the 16th greatest British writer since 1945., Roald Dahl is British.}",Roald Dahl,"Roald Dahl ([langpronˈroʊ.əld ˈdɑːl], [ˈɾuːɑl dɑl]; 13 September 1916 - 23 November 1990) was a British novelist, short story writer, poet, screenwriter, and fighter pilot. His books have sold more than 250 million copies worldwide.","Roald Dahl ([langpronˈroʊ.əld _ ˈdɑːl], [ˈɾuːɑl dɑl]; 13 September 1916 - 23 November 1990) was a British novelist, short story writer, poet, screenwriter, and fighter pilot. His books have sold more than 250 million copies worldwide. Born in Wales to Norwegian parents, Dahl served in the Royal Air Force during the Second World War, in which he became a flying ace and intelligence officer, rising to the rank of acting wing commander. He rose to prominence in the 1940s with works for both children and ...","[Christopher T. Sununu -LSB- səˈnuːnuː -RSB- -LRB- born November 5 , 1974 -RRB- is an American politician and businessman who is the 82nd and current Governor of New Hampshire , since January 2017 . Sununu was previously a member of the New Hampshire Executive Council , an office he held, Paul Richard LePage -LSB- ləˈpeɪdʒ -RSB- -LRB- born October 9 , 1948 -RRB- is an American businessman and Republican Party politician who is currently the 74th Governor of Maine . Born in Lewiston , LePage grew up with...","[Chris Sununu, Paul LePage, Roald Dahl, Roald Dahl, Roald Dahl, Roald Dahl, Roald Dahl bibliography, Roald Dahl, Roald Dahl, Roald Dahl, Roald Dahl, Roald Dahl, Roald Dahl, Roald Dahl, Roald Dahl, Roald Dahl, Roald Dahl, Roald Dahl, Roald Dahl, Roald Dahl, Roald Dahl, Roald Dahl, Roald Dahl, Roald Dahl]"
2,5,"{Ireland has a central plain., Ireland has relatively low-lying mountains., Ireland is

In [169]:
df_grp["pipeline_sents"] = df_grp["pipeline_sents"].apply(lambda x: [normalize(el) for el in x])

In [170]:
# copy source_sentence to df_grp2, as source_sentence_id
df_grp2["source_sentence_id"] = df_grp["source_sentence"]
df_grp2["pipeline_sents"] = df_grp["pipeline_sents"]

In [208]:
# in source_text replace ".   " with ".\n"
df_grp["source_text"] = df_grp["source_text"].apply(lambda x: x.replace(".   ", ".\n"))

/tmp/ipykernel_1240981/998644743.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_grp["source_text"] = df_grp["source_text"].apply(lambda x: x.replace(".   ", ".\n"))


In [210]:
df_grp["sentence_context"] = df_grp["sentence_context"].apply(lambda x: [el.strip() for el in x])

/tmp/ipykernel_1240981/4092188698.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_grp["sentence_context"] = df_grp["sentence_context"].apply(lambda x: [el.strip() for el in x])


In [211]:
df_grp["sentence_context"]

0       [Chris Hemsworth, He is known for playing Kim Hyde in the Australian TV series Home and Away (2004-07) and Thor in the Marvel Cinematic Universe since 2011., Hemsworth has also appeared in the science fiction action film Star Trek (2009), the thriller adventure A Perfect Getaway (2009), the horror comedy The Cabin in the Woods (2012), the dark-fantasy action film Snow White and the Huntsman (2012), the war film Red Dawn (2012), and the biographical sports drama film Rush (2013)., In 2015, he starred in ...
1                                 [Roald Dahl, His awards for contribution to literature include the 1983 World Fantasy Award for Life Achievement, and the British Book Awards' Children's Author of the Year in 1990., In 2008, The Times placed Dahl 16th on its list of "The 50 greatest British writers since 1945"., Dahl's short stories are known for their unexpected endings and his children's books for their unsentimental, macabre, often darkly comic mood, featuring villainous 

In [35]:
# cast claims to list
df_grp["claims"] = df_grp["claims"].apply(list)

## Instructor 👨‍🏫 Expts

In [39]:
import pysbd
# segment the source_text
seg = pysbd.Segmenter(language="en", clean=False)

In [103]:
df_grp["context"] = None

In [173]:
df_grp = df_grp2
df_grp.sample(5)

,source_sentence,claims,source,source_sent,source_text,pipeline_text,source_sents,source_ngram,source_sentence_id,pipeline_sents
66,"Cindy Lou Hensley McCain (born May 20, 1954) is an American businesswoman, philanthropist, and humanitarian, and the wife of long-time United States Senator and 2008 Republican presidential nominee John McCain of Arizona.","[Cindy McCain is married to a 2008 presidential nominee., Cindy McCain is a philanthropist., Cindy McCain is a businesswoman., Cindy McCain is American., Cindy McCain is married to John McCain.]",Cindy McCain,"be elected to the United States Congress the following year and has been in office since then. The couple had three children together, in addition to adopting another. From 1988 to 1995, she founded and operated a nonprofit organization, the American Voluntary Medical Team,","Cindy Lou Hensley McCain (born May 20, 1954) is an American businesswoman, philanthropist, and humanitarian, and the wife of long-time United States Senator and 2008 Republican presidential nominee John McCain of Arizona. She was born and raised in Phoenix, Arizona, as the daughter of wealthy beer distributor Jim Hensley. After receiving bachelor's and master's degrees from the University of Southern California, she became a special education teacher. She married John McCain in 1980 and the couple mov...","[Cindy McCain, Cindy McCain, Cindy McCain, Cindy McCain, Cindy McCain, Cindy McCain, Cindy McCain, Cindy McCain, Cindy McCain, Cindy McCain, Cindy McCain, Cindy McCain, Cindy McCain, Cindy McCain, Cindy McCain, Cindy McCain, Cindy McCain, Cindy McCain, Cindy McCain, Cindy McCain, Cindy McCain, Cindy McCain, Cindy McCain, Cindy McCain, Cindy McCain, Cindy McCain, Cindy McCain, Cindy McCain, Cindy McCain, Cindy McCain, Cindy McCain, Cindy McCain, Cindy McCain, Cindy McCain, Cindy McCain, Cindy McCain]","[Cindy Lou Hensley McCain (born May 20, 1954) is an American businesswoman, philanthropist, and humanitarian, and the wife of long-time United States Senator and 2008 Republican presidential nominee John McCain of Arizona. , She was born and raised in Phoenix, Arizona, as the daughter of wealthy beer distributor Jim Hensley. , After receiving bachelor's and master's degrees from the University of Southern California, she became a special education teacher. , She married John McCain in 1980 and the cou...","[Cindy McCain, Cindy Lou Hensley McCain (born May 20, 1954) is an American businesswoman, philanthropist, and humanitarian, and the wife of long-time United States Senator and 2008 Republican presidential nominee John McCain of Arizona. , She was born and raised in Phoenix, Arizona, as the daughter of wealthy beer distributor Jim Hensley. ]",83,"[the daughter of wealthy beer distributor Jim Hensley. After receiving bachelor's and master's degrees from the University of Southern California, she became a special education teacher. She married John McCain in 1980 and the couple moved to Arizona in 1981, where her husband would, be elected to the United States Congress the following year and has been in office since then. The couple had three children together, in addition to adopting another. From 1988 to 1995, she founded and operated a nonprofit..."
407,"Varicella zoster virus is not the same as herpes simplex virus; however, they belong to the same family of viruses.","[Shingles belong to the same family of viruses herpes simplex is part of., Shingles is a virus that is not like herpes simplex virus.]",Shingles,"one attack. The disease has been recognized since ancient times. Shingles, also known as herpes zoster, is a viral disease characterized by a painful skin rash with blisters in a localized area. Typically the rash occurs in a single stripe either on the left or","Shingles, also known as herpes zoster, is a viral disease characterized by a painful skin rash with blisters in a localized area. Typically the rash occurs in a single stripe either on the left or right of the body or face. Two to 

In [105]:
df_grp["context"][:631]=nearest_ngrams

/tmp/ipykernel_1240981/1959592446.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_grp["context"][:631]=nearest_ngrams


In [108]:
len(ngrams), len(source_sents)

(6, 6)

In [122]:
%pip install angle-emb

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Obtaining dependency information for angle-emb from https://files.pythonhosted.org/packages/a1/6d/00cab5f0ab7df0ae800404e6fa3717be5eac78284550c300e7e7bddadf4a/angle_emb-0.3.3-py3-none-any.whl.metadata
  Using cached angle_emb-0.3.3-py3-none-any.whl.metadata (15 kB)
Using cached angle_emb-0.3.3-py3-none-any.whl (39 kB)

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [123]:
from angle_emb import AnglE, Prompts

angle = AnglE.from_pretrained('WhereIsAI/UAE-Large-V1', pooling_strategy='cls').cuda()
angle.set_prompt(prompt=Prompts.C)
vec = angle.encode({'text': 'hello world'}, to_numpy=True)
print(vec)
vecs = angle.encode([{'text': 'hello world1'}, {'text': 'hello world2'}], to_numpy=True)
print(vecs)

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/733 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

INFO:AnglE:Prompt is set, the prompt will be automatically applied during the encoding phase. To disable prompt setting, please configure set_prompt(prompt=None)


[[ 0.37528038  0.08762033  0.5270614  ... -0.26766053  0.0266216
  -0.14156047]]
[[ 0.49360007  0.16666055  0.46393126 ... -0.16278444 -0.10181473
  -0.23928286]
 [ 0.2607215   0.63582     0.88706267 ... -0.40785685 -0.25375688
  -0.15815154]]


In [130]:
similarities

array([[0.77114224],
       [0.5669539 ],
       [0.6873406 ],
       [0.47543442],
       [0.42952704]], dtype=float32)

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/568M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/18.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

/home/ullriher/venvs/2023feb/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:515: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
INFO:sentence_transformers.cross_encoder.CrossEncoder:Use pytorch device: cuda


In [144]:
from sentence_transformers import CrossEncoder
model = CrossEncoder('cross-encoder/nli-deberta-v3-small')

scores = model.predict([('A man is eating pizza', 'A man eats something'), ('A black race car starts up in front of a crowd of people.', 'A man is driving down a lonely road.')], apply_softmax=True)
scores

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

array([[2.4521039e-04, 9.6874678e-01, 3.1008026e-02],
       [9.9972779e-01, 6.8679816e-05, 2.0350475e-04]], dtype=float32)

In [145]:
# get only column 1 from scores
scores = scores[:,1]
scores

array([9.6874678e-01, 6.8679816e-05], dtype=float32)

In [141]:
label_mapping = ['contradiction', 'entailment', 'neutral']
labels = [u[1] for u in scores]
labels

[0.9687468, 6.8679816e-05]

In [152]:
df_grp.head(10)

,source_sentence,claims,source,source_sent,source_text,pipeline_text,source_sents,context
0,2,"[Chris Hemsworth appeared in A Perfect Getaway., Chris Hemsworth appeared in Star Trek., Chris Hemsworth is known for his role on the Australian TV series Home and Away., Chris Hemsworth is known for playing Thor in The Marvel Cinematic Universe., Chris Hemsworth is known for playing Kim Hyde.]",Chris Hemsworth,Chris Hemsworth (born 11 August 1983) is an Australian actor. He is known for playing Kim Hyde in the Australian TV series Home and Away (2004-07) and Thor in the Marvel Cinematic Universe since 2011. Hemsworth has also appeared in the science fiction action,"Chris Hemsworth (born 11 August 1983) is an Australian actor. He is known for playing Kim Hyde in the Australian TV series Home and Away (2004-07) and Thor in the Marvel Cinematic Universe since 2011. Hemsworth has also appeared in the science fiction action film Star Trek (2009), the thriller adventure A Perfect Getaway (2009), the horror comedy The Cabin in the Woods (2012), the dark-fantasy action film Snow White and the Huntsman (2012), the war film Red Dawn (2012), and the biographical sports drama...","[Kevin Costner, Seth Gilliam, Shane Withington, Adam Willits, Chris Hemsworth, Chris Hemsworth, Thor (film), Rick Cosnett, T.J. Lowther, Donald Faison, Chris Jarvis (presenter), Chris Jarvis (presenter), Marshall Lancaster, Chris Hemsworth, Adam Willits, Shane Withington, Donald Faison, Chris Jarvis (presenter), Chris Hemsworth, Thor (film), Peter Cundall, Sam Kekovich, Chris Lilley (comedian), John Corbett, A Perfect Getaway, A Perfect Getaway, John Corbett, Donald Faison, Chris Hemsworth, A Perfect Ge...","[Chris Hemsworth (born 11 August 1983) is an Australian actor. , He is known for playing Kim Hyde in the Australian TV series Home and Away (2004-07) and Thor in the Marvel Cinematic Universe since 2011. , Hemsworth has also appeared in the science fiction action film Star Trek (2009), the thriller adventure A Perfect Getaway (2009), the horror comedy The Cabin in the Woods (2012), the dark-fantasy action film Snow White and the Huntsman (2012), the war film Red Dawn (2012), and the biographical sports ...","[Chris Hemsworth, Chris Hemsworth (born 11 August 1983) is an Australian actor. , He is known for playing Kim Hyde in the Australian TV series Home and Away (2004-07) and Thor in the Marvel Cinematic Universe since 2011. , Hemsworth has also appeared in the science fiction action film Star Trek (2009), the thriller adventure A Perfect Getaway (2009), the horror comedy The Cabin in the Woods (2012), the dark-fantasy action film Snow White and the Huntsman (2012), the war film Red Dawn (2012), and the bio..."
1,4,"[Roald Dahl is a writer., Roald Dahl was listed as the 16th greatest British writer since 1945., Roald Dahl is British.]",Roald Dahl,"Roald Dahl ([langpronˈroʊ.əld ˈdɑːl], [ˈɾuːɑl dɑl]; 13 September 1916 - 23 November 1990) was a British novelist, short story writer, poet, screenwriter, and fighter pilot. His books have sold more than 250 million copies worldwide.","Roald Dahl ([langpronˈroʊ.əld _ ˈdɑːl], [ˈɾuːɑl dɑl]; 13 September 1916 - 23 November 1990) was a British novelist, short story writer, poet, screenwriter, and fighter pilot. His books have sold more than 250 million copies worldwide. Born in Wales to Norwegian parents, Dahl served in the Royal Air Force during the Second World War, in which he became a flying ace and intelligence officer, rising to the rank of acting wing commander. He rose to prominence in the 1940s with works for both children and ...","[Chris Sununu, Paul LePage, Roald Dahl, Roald Dahl, Roald Dahl, Roald Dahl, Roald Dahl bibliography, Roald Dahl, Roald Dahl, Roald Dahl, Roald Dahl, Roald Dahl, Roald Dahl, Roald Dahl, Roald Dahl, Roald Dahl, Roald Dahl, Roald Dahl, Roald Dahl, Roald Dahl, Roald Dahl, Roald Dahl, Roald Dahl, Roald Dahl]","[Roald Dahl ([langpronˈroʊ.əld _ ˈdɑːl], [ˈɾuːɑl dɑl]; 13 September 1916 - 23 Novemb

In [154]:
#drop context column
if "context" in df_grp.columns:
    df_grp.drop(columns=["context"], inplace=True)



In [338]:
df_grp.sample(3)

,sentence_id,source,sentence,claims,source_text,sentence_context,source_sentence,source_ngram
2145,2853,Jensen Ackles,"He is known for his roles in television as Eric Brady in Days of Our Lives, which earned him several Daytime Emmy Award nominations, as well as Alec/X5 -494 in Dark Angel and Jason Teague in Smallville.","[Jensen Ackles played the lead in My Bloody Valentine 3D., Jensen Ackles played Jason Teague in Smallville., Jensen Ackles played Alec/X5-494 in Dark Angel., Jensen Ackles received several Daytime Emmy Award nominations., Jensen Ackles played Eric Brady in Days of Our Lives.]","Jensen Ackles\nJensen Ross Ackles (born March 1, 1978) is an American actor and director. He is known for his roles in television as Eric Brady in Days of Our Lives, which earned him several Daytime Emmy Award nominations, as well as Alec/X5 -494 in Dark Angel and Jason Teague in Smallville. He also starred as the lead in the box office success My Bloody Valentine 3D and portrayed Jason Todd in the popular animated film Batman: Under the Red Hood.\nAckles currently stars as Dean Winchester on the CW ser...","[Jensen Ackles, Jensen Ross Ackles (born March 1, 1978) is an American actor and director., He is known for his roles in television as Eric Brady in Days of Our Lives, which earned him several Daytime Emmy Award nominations, as well as Alec/X5 -494 in Dark Angel and Jason Teague in Smallville., He also starred as the lead in the box office success My Bloody Valentine 3D and portrayed Jason Todd in the popular animated film Batman: Under the Red Hood.]",None,None
4480,6352,Mukesh Ambani,"Mukesh Dhirubhai Ambani (born 19 April 1957) is an Indian business magnate who is the chairman, managing director and largest shareholder of Reliance Industries Limited (RIL), a Fortune Global 500 company and India's second-most valuable company by market value.","[Mukesh Ambani is the son of the founder of Reliance Industries., Mukesh Ambani is the brother of an Indian business magnate.]","Mukesh Ambani\nMukesh Dhirubhai Ambani (born 19 April 1957) is an Indian business magnate who is the chairman, managing director and largest shareholder of Reliance Industries Limited (RIL), a Fortune Global 500 company and India's second-most valuable company by market value. He holds a 44.7 % stake in the company. RIL deals mainly in refining, petrochemicals, and in the oil and gas sectors. Reliance Retail Ltd., another subsidiary, is the largest retailer in India.\nHe is the elder son of the late Dhi...","[Mukesh Ambani, Mukesh Dhirubhai Ambani (born 19 April 1957) is an Indian business magnate who is the chairman, managing director and largest shareholder of Reliance Industries Limited (RIL), a Fortune Global 500 company and India's second-most valuable company by market value., He holds a 44.7 % stake in the company.]",None,None
1629,2180,Teen Wolf (2011 TV series),"It is loosely based on the 1985 film of the same name, and stars Tyler Posey as a teenager named Scott McCall, who is bitten by a werewolf and must cope with how it affects his life and the lives of those closest to him, and Dylan O'Brien as ""Stiles"" Stilinski, Scott's best friend.","[Teen Wolf follows the story of a teenager named Scott McCall, who is bitten by a werewolf and must cope with how it affects his life., Teen Wolf stars Tyler Posey., Teen Wolf was developed by Jeff Davis., Teen Wolf is an American television series., Teen Wolf is loosely based on the 1985 film of the same name.]","Teen Wolf (2011 TV series)\nTeen Wolf is an American television series developed by Jeff Davis for MTV. It is loosely based on the 1985 film of the same name, and stars Tyler Posey as a teenager named Scott McCall, who is bitten by a werewolf and must cope with how it affects his life and the lives of those closest to him, and Dylan O'Brien as ""Stiles"" Stilinski, Scott's best friend. The series has received generally positive reviews from critics and is a fan favorite on social media.\nTeen Wolf premier

In [339]:
import pysbd
# segment the source_text
seg = pysbd.Segmenter(language="en", clean=False)
df_grp["source_sents"]=df_grp["source_text"].apply(lambda x: seg.segment(x))

/tmp/ipykernel_1240981/4190464788.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_grp["source_sents"]=df_grp["source_text"].apply(lambda x: seg.segment(x))


In [340]:
import numpy as np
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

# Create an empty list to store the nearest ngrams
nearest_ngrams = []
df_grp["source_sentence"] = None
df_grp["source_ngram"] = None

# Iterate over each row in df_grp
for i in tqdm(range(len(df_grp))):
    # Get the source_sents for the current row
    source_sents = df_grp.iloc[i]["source_sents"]
    
    # Form 3-grams
    ngrams = []
    for j in range(len(source_sents)):
        ngrams.append([df_grp.iloc[i]["source"]] + source_sents[max(j-1, 0):j+2])
    
    # Initialize ngram_scores as a NumPy array
    ngram_scores = np.zeros(len(ngrams))
    
    # Encode ngrams and claims
    claims = df_grp.iloc[i]["claims"]
    for claim in claims:
        ngram_scores += model.predict(list(zip([df_grp.iloc[i]["source"]+": "+s for s in source_sents], [claim]*len(ngrams))), apply_softmax=True, show_progress_bar=False)[:,1]
    
    
    # Get the index of the nearest ngram
    nearest_index = np.argmax(ngram_scores)
    if ngram_scores[nearest_index] < 0.5:
        df_grp["source_sentence"][i] = None
        df_grp["source_ngram"][i] = None
    else:
        df_grp["source_sentence"][i] = source_sents[nearest_index]
        df_grp["source_ngram"][i] = ngrams[nearest_index]

# Print the modified DataFrame
df_grp


/tmp/ipykernel_1240981/1085048450.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_grp["source_sentence"] = None
/tmp/ipykernel_1240981/1085048450.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_grp["source_ngram"] = None
  0%|          | 0/4585 [00:00<?, ?it/s]/tmp/ipykernel_1240981/1085048450.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

,sentence_id,source,sentence,claims,source_text,sentence_context,source_sentence,source_ngram,source_sents
0,2,Chris Hemsworth,"Hemsworth has also appeared in the science fiction action film Star Trek (2009), the thriller adventure A Perfect Getaway (2009), the horror comedy The Cabin in the Woods (2012), the dark-fantasy action film Snow White and the Huntsman (2012), the war film Red Dawn (2012), and the biographical sports drama film Rush (2013).","[Chris Hemsworth appeared in A Perfect Getaway., Chris Hemsworth appeared in Star Trek., Chris Hemsworth is known for his role on the Australian TV series Home and Away., Chris Hemsworth is known for playing Thor in The Marvel Cinematic Universe., Chris Hemsworth is known for playing Kim Hyde.]","Chris Hemsworth\nChris Hemsworth (born 11 August 1983) is an Australian actor. He is known for playing Kim Hyde in the Australian TV series Home and Away (2004-07) and Thor in the Marvel Cinematic Universe since 2011. Hemsworth has also appeared in the science fiction action film Star Trek (2009), the thriller adventure A Perfect Getaway (2009), the horror comedy The Cabin in the Woods (2012), the dark-fantasy action film Snow White and the Huntsman (2012), the war film Red Dawn (2012), and the biograph...","[Chris Hemsworth, He is known for playing Kim Hyde in the Australian TV series Home and Away (2004-07) and Thor in the Marvel Cinematic Universe since 2011., Hemsworth has also appeared in the science fiction action film Star Trek (2009), the thriller adventure A Perfect Getaway (2009), the horror comedy The Cabin in the Woods (2012), the dark-fantasy action film Snow White and the Huntsman (2012), the war film Red Dawn (2012), and the biographical sports drama film Rush (2013)., In 2015, he starred in ...","Hemsworth has also appeared in the science fiction action film Star Trek (2009), the thriller adventure A Perfect Getaway (2009), the horror comedy The Cabin in the Woods (2012), the dark-fantasy action film Snow White and the Huntsman (2012), the war film Red Dawn (2012), and the biographical sports drama film Rush (2013).\n","[Chris Hemsworth, He is known for playing Kim Hyde in the Australian TV series Home and Away (2004-07) and Thor in the Marvel Cinematic Universe since 2011. , Hemsworth has also appeared in the science fiction action film Star Trek (2009), the thriller adventure A Perfect Getaway (2009), the horror comedy The Cabin in the Woods (2012), the dark-fantasy action film Snow White and the Huntsman (2012), the war film Red Dawn (2012), and the biographical sports drama film Rush (2013).\n, In 2015, he starred ...","[Chris Hemsworth\n, Chris Hemsworth (born 11 August 1983) is an Australian actor. , He is known for playing Kim Hyde in the Australian TV series Home and Away (2004-07) and Thor in the Marvel Cinematic Universe since 2011. , Hemsworth has also appeared in the science fiction action film Star Trek (2009), the thriller adventure A Perfect Getaway (2009), the horror comedy The Cabin in the Woods (2012), the dark-fantasy action film Snow White and the Huntsman (2012), the war film Red Dawn (2012), and the b..."
1,4,Roald Dahl,"In 2008, The Times placed Dahl 16th on its list of ""The 50 greatest British writers since 1945"".","[Roald Dahl is a writer., Roald Dahl was listed as the 16th greatest British writer since 1945., Roald Dahl is British.]","Roald Dahl\nRoald Dahl ([langpronˈroʊ.əld _ ˈdɑːl], [ˈɾuːɑl dɑl]; 13 September 1916 - 23 November 1990) was a British novelist, short story writer, poet, screenwriter, and fighter pilot. His books have sold more than 250 million copies worldwide.\nBorn in Wales to Norwegian parents, Dahl served in the Royal Air Force during the Second World War, in which he became a flying ace and intelligence officer, rising to the rank of acting wing commander. He rose to prominence in the 1940s with works for both ch...","[Roald Dahl, His awards for contribution to literature include the 1983 World Fantasy Award for Life Achiev

In [341]:
# count ones with none as source_sentence
len(df_grp[df_grp["source_sentence"].isna()])

166

In [342]:
# now toss them
df_grp = df_grp[~df_grp["source_sentence"].isna()]

In [343]:
len(df_grp)

4419

In [ ]:
#4585

In [ ]:
df_grp_backup = df_grp.copy()
# move sent id up front and remove pipeline cols: source_sentence	claims	source	source_sent	source_text	pipeline_text	source_sents	source_ngram	source_sentence_id
df_grp = df_grp[["source_sentence_id", "source", "source_sentence",  "claims", "source_text", "source_ngram"]]

In [190]:
# rename ngram to context, rename source_text to full_text, source_sentence_id to sentence_id, source_sentence to sentence
df_grp.rename(columns={"source_sentence_id": "sentence_id", "source_sentence": "sentence", "source_ngram": "context", "source_text": "full_text"}, inplace=True)

/tmp/ipykernel_1240981/3740802683.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_grp.rename(columns={"source_sentence_id": "sentence_id", "source_sentence": "sentence", "source_ngram": "context", "source_text": "full_text"}, inplace=True)


In [199]:
#rename full_text back to source_text
df_grp.rename(columns={"full_text": "source_text"}, inplace=True)

/tmp/ipykernel_1240981/3482988040.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_grp.rename(columns={"full_text": "source_text"}, inplace=True)


In [201]:
# rename context to sentence_context
df_grp.rename(columns={"context": "sentence_context"}, inplace=True)

/tmp/ipykernel_1240981/1765868857.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_grp.rename(columns={"context": "sentence_context"}, inplace=True)


In [354]:
df_grp.sample(3)

,sentence_id,source,sentence,claims,source_text,sentence_context,source_sentence,source_ngram,source_sents
1726,2313,Hot Right Now,"""Hot Right Now"" is a single by British drum and bass producer DJ Fresh, released as the second single from his third studio album, Nextlevelism.","[Hot Right Now is from the album Nextlevelism., Hot Right Now is a DJ Fresh single., Hot Right Now is off of DJ Fresh's third album.]","Hot Right Now\n""Hot Right Now"" is a single by British drum and bass producer DJ Fresh, released as the second single from his third studio album, Nextlevelism. British singer Rita Ora features on the single as the main vocalist. The song also features as a bonus track on Ora's debut album, Ora. Released on 12 February 2012, the single debuted at the top of UK Singles Chart. The Official Charts Company confirmed ""Hot Right Now"" as the UK's first ever drum and bass number one.","[Hot Right Now, ""Hot Right Now"" is a single by British drum and bass producer DJ Fresh, released as the second single from his third studio album, Nextlevelism., British singer Rita Ora features on the single as the main vocalist.]","""Hot Right Now"" is a single by British drum and bass producer DJ Fresh, released as the second single from his third studio album, Nextlevelism.","[Hot Right Now, Hot Right Now\n, ""Hot Right Now"" is a single by British drum and bass producer DJ Fresh, released as the second single from his third studio album, Nextlevelism. , British singer Rita Ora features on the single as the main vocalist. ]","[Hot Right Now\n, ""Hot Right Now"" is a single by British drum and bass producer DJ Fresh, released as the second single from his third studio album, Nextlevelism. , British singer Rita Ora features on the single as the main vocalist. , The song also features as a bonus track on Ora's debut album, Ora. , Released on 12 February 2012, the single debuted at the top of UK Singles Chart. , The Official Charts Company confirmed ""Hot Right Now"" as the UK's first ever drum and bass number one. ]"
802,1066,1992 Los Angeles riots,"The 1992 Los Angeles riots, also known as the Rodney King riots, the South Central riots, the 1992 Los Angeles civil disturbance, the 1992 Los Angeles civil unrest, and the Los Angeles uprising, were a series of riots, lootings, arsons, and civil disturbances that occurred in Los Angeles County, California in April and May 1992.","[The 1992 Los Angeles riots have many names., The 1992 Los Angeles riots occurred in the most populous county in the United States., The 1992 Los Angeles riots were a series of events of civil disorder., The 1992 Los Angeles riots occurred in California., The 1992 Los Angeles riots occurred in 1992.]","1992 Los Angeles riots\nThe 1992 Los Angeles riots, also known as the Rodney King riots, the South Central riots, the 1992 Los Angeles civil disturbance, the 1992 Los Angeles civil unrest, and the Los Angeles uprising, were a series of riots, lootings, arsons, and civil disturbances that occurred in Los Angeles County, California in April and May 1992. The unrest began in South Central Los Angeles on April 29 after a trial jury acquitted four officers of the Los Angeles Police Department of the use of e...","[1992 Los Angeles riots, The 1992 Los Angeles riots, also known as the Rodney King riots, the South Central riots, the 1992 Los Angeles civil disturbance, the 1992 Los Angeles civil unrest, and the Los Angeles uprising, were a series of riots, lootings, arsons, and civil disturbances that occurred in Los Angeles County, California in April and May 1992., The unrest began in South Central Los Angeles on April 29 after a trial jury acquitted four officers of the Los Angeles Police Department of the use of...","The 1992 Los Angeles riots, also known as the Rodney King riots, the South Central riots, the 1992 Los Angeles civil disturbance, the 1992 Los Angeles civil unrest, and the Los Angeles uprising, were a series of riots, lootings, arsons, and civil disturbances t

In [196]:
df_grp.iloc[536]

sentence_id                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                708
source                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [356]:
print("čáu")

čáu


In [67]:
nearest

0

In [68]:
claims

['Roald Dahl is a writer.',
 'Roald Dahl was listed as the 16th greatest British writer since 1945.',
 'Roald Dahl is British.']

In [38]:
embeddings

array([[-0.01202731,  0.00817354,  0.02546182, ..., -0.03163337,
        -0.01569521,  0.09189018],
       [ 0.03912366,  0.03356262,  0.01438785, ..., -0.03231231,
         0.01382168,  0.08592679],
       [ 0.01773507,  0.04967228,  0.02618054, ..., -0.03792765,
        -0.01430106,  0.09450648],
       [ 0.02169752,  0.00533172,  0.00854104, ..., -0.04038185,
        -0.01339713,  0.09915996],
       [ 0.00246993,  0.03627908,  0.00098812, ..., -0.03143589,
        -0.03131983,  0.0734854 ]], dtype=float32)

In [203]:
# prepend source \n to source_text
df_grp["source_text"] = df_grp.apply(lambda x: f"{x['source']}\n{x['source_text']}", axis=1)

/tmp/ipykernel_1240981/2547822022.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_grp["source_text"] = df_grp.apply(lambda x: f"{x['source']}\n{x['source_text']}", axis=1)


In [204]:
print(wiki["Roald Dahl"])

Roald Dahl ([langpronˈroʊ.əld _ ˈdɑːl], [ˈɾuːɑl dɑl]; 13 September 1916 - 23 November 1990) was a British novelist, short story writer, poet, screenwriter, and fighter pilot. His books have sold more than 250 million copies worldwide.   Born in Wales to Norwegian parents, Dahl served in the Royal Air Force during the Second World War, in which he became a flying ace and intelligence officer, rising to the rank of acting wing commander. He rose to prominence in the 1940s with works for both children and adults and he became one of the world's best selling authors. He has been referred to as "one of the greatest storytellers for children of the 20th century". His awards for contribution to literature include the 1983 World Fantasy Award for Life Achievement, and the British Book Awards' Children's Author of the Year in 1990. In 2008, The Times placed Dahl 16th on its list of "The 50 greatest British writers since 1945".   Dahl's short stories are known for their unexpected endings and hi

In [205]:
print(df_grp.iloc[537]["source_text"])

Russia
Russia ([ˈrʌʃə] [Россия, Rossiya, rɐˈsʲijə]), also officially the Russian Federation ([Российская Федерация, r = Rossiyskaya Federatsiya, p = rɐˈsʲijskəjə fʲɪdʲɪˈratsɨjə]), is a country in Eurasia. At 17075200 km2, Russia is the largest country in the world by surface area, covering more than one-eighth of the Earth's inhabited land area, and the ninth most populous, with over 144 million people at the end of March 2016. The European western part of the country is much more populated and urbanised than the eastern; about 77 % of the population live in European Russia. Russia's capital Moscow is one of the largest cities in the world; other major urban centers include Saint Petersburg, Novosibirsk, Yekaterinburg, Nizhny Novgorod and Kazan.   Extending across the entirety of Northern Asia and much of Eastern Europe, Russia spans eleven time zones and incorporates a wide range of environments and landforms. From northwest to southeast, Russia shares land borders with Norway, Finlan

In [198]:
dict(df_grp.iloc[4527])
df_grp.iloc[536]["source_text"]

KeyError: 'source_text'

In [80]:
new_source= normalize("Wikipedia")
new_source_text = normalize("Wikipedia\nWikipedia ([wɪkiˈpiːdiə]) is a free online encyclopedia with the aim to allow anyone to edit articles. Wikipedia is the largest and most popular general reference work on the Internet and is ranked among the ten most popular websites. Wikipedia is owned by the nonprofit Wikimedia Foundation.\nWikipedia was launched on January 15, 2001, by Jimmy Wales and Larry Sanger. Sanger coined Wikipedia, a portmanteau of wiki and encyclopedia. There was only the English language version initially, but it quickly developed similar versions in other languages, which differ in content and in editing practices. With articles, the English Wikipedia is the largest of the more than 290 Wikipedia encyclopedias. Overall, Wikipedia consists of more than 40 million articles in more than 250 different languages and,, it had 18 billion page views and nearly 500 million unique visitors each month.\nAs of March 2017, Wikipedia has about forty thousand high-quality articles known as Featured Articles and Good Articles that cover vital topics. In 2005, Nature published a peer review comparing 42 science articles from Encyclopædia Britannica and Wikipedia, and found that Wikipedia\'s level of accuracy approached Encyclopædia Britannicas. Criticism of Wikipedia includes claims that it exhibits systemic bias, presents a mixture of \"truths, half truths, and some falsehoods\", and that, in controversial topics, it is subject to manipulation and spin.", False)
# assign to 536th row
df_grp.at[536, "source"] = new_source
df_grp.at[536, "source_text"] = new_source_text

In [60]:
wiki["Wikipedia"]

'Wikipedia ([wɪkiˈpiːdiə]) is a free online encyclopedia with the aim to allow anyone to edit articles. Wikipedia is the largest and most popular general reference work on the Internet and is ranked among the ten most popular websites. Wikipedia is owned by the nonprofit Wikimedia Foundation.   Wikipedia was launched on January 15, 2001, by Jimmy Wales and Larry Sanger. Sanger coined Wikipedia, a portmanteau of wiki and encyclopedia. There was only the English language version initially, but it quickly developed similar versions in other languages, which differ in content and in editing practices. With articles, the English Wikipedia is the largest of the more than 290 Wikipedia encyclopedias. Overall, Wikipedia consists of more than 40 million articles in more than 250 different languages and,, it had 18 billion page views and nearly 500 million unique visitors each month.   As of March 2017, Wikipedia has about forty thousand high-quality articles known as Featured Articles and Good 

In [57]:
print(len(wiki))

5416535


In [357]:
df_grp.sample(13)

,sentence_id,source,sentence,claims,source_text,sentence_context,source_sentence,source_ngram,source_sents
1659,2218,Blue Velvet (film),It nevertheless earned Lynch his second Academy Award nomination for Best Director and came to achieve cult status.,"[Blue Velvet has cult status., Blue Velvet was nominated., Blue Velvet is a film.]","Blue Velvet (film)\nBlue Velvet is a 1986 American neo-noir mystery film, written and directed by David Lynch. Blending psychological horror with film noir, the film stars Kyle MacLachlan, Isabella Rossellini, Dennis Hopper and Laura Dern. The title is taken from Bobby Vinton's 1963 song of the same name.\nThe screenplay of Blue Velvet had been passed around multiple times in the late 1970s and early 1980s, with many major studios declining it because of its strong sexual and violent content. After the ...","[Blue Velvet (film), Blue Velvet initially received a divided critical response, with many stating that its objectionable content served little artistic purpose., It nevertheless earned Lynch his second Academy Award nomination for Best Director and came to achieve cult status., As an example of a director casting against the norm, it was credited for re-launching Hopper's career and for providing Rossellini with a dramatic outlet beyond her previous work as a fashion model and a cosmetics spokeswoman.]",It nevertheless earned Lynch his second Academy Award nomination for Best Director and came to achieve cult status.,"[Blue Velvet (film), Blue Velvet initially received a divided critical response, with many stating that its objectionable content served little artistic purpose. , It nevertheless earned Lynch his second Academy Award nomination for Best Director and came to achieve cult status. , As an example of a director casting against the norm, it was credited for re-launching Hopper's career and for providing Rossellini with a dramatic outlet beyond her previous work as a fashion model and a cosmetics spokeswoman...","[Blue Velvet (film)\n, Blue Velvet is a 1986 American neo-noir mystery film, written and directed by David Lynch. , Blending psychological horror with film noir, the film stars Kyle MacLachlan, Isabella Rossellini, Dennis Hopper and Laura Dern. , The title is taken from Bobby Vinton's 1963 song of the same name.\n, The screenplay of Blue Velvet had been passed around multiple times in the late 1970s and early 1980s, with many major studios declining it because of its strong sexual and violent content. ,..."
1695,2273,Marilyn Monroe,"On August 5, 1962, she died at age 36 from an overdose of barbiturates at her home in Los Angeles.","[Marilyn Monroe died in 1962., Marilyn Monroe died of a drug overdose., Marilyn Monroe died in Los Angeles., Marilyn Monroe died of a barbiturates overdose., Marilyn Monroe died on August the 5th, 1962.]","Marilyn Monroe\nMarilyn Monroe (born Norma Jeane Mortenson; June 1, 1926 - August 5, 1962) was an American actress and model. Famous for playing comic ""dumb blonde"" characters, she became one of the most popular sex symbols of the 1950s, emblematic of the era's attitudes towards sexuality. Although she was a top-billed actress for only a decade, her films grossed $ 200 million by the time of her unexpected death in 1962. She continues to be considered a major popular culture icon.\nBorn and raised in Lo...","[Marilyn Monroe, She had two highly publicized marriages, to retired baseball star Joe DiMaggio and playwright Arthur Miller, both of which ended in divorce., On August 5, 1962, she died at age 36 from an overdose of barbiturates at her home in Los Angeles., Although Monroe's death was ruled a probable suicide, several conspiracy theories have been proposed in the decades following her death.]","On August 5, 1962, she died at age 36 from an overdose of barbiturates at her home in Los Angeles.","[Marilyn Monroe, She had two highly publicized marriages, to retired baseball star Joe DiMaggio and playwright Arthur Miller, both of which ended

In [347]:
df_grp.iloc[2572]["source_text"]

"Bridget Moynahan\nBridget Moynahan (born Kathryn Bridget Moynahan; April 28, 1971) is an American actress and model. She graduated from Longmeadow High School, in Massachusetts, in 1989, and began pursuing a career in modeling. She appeared in department-store catalogs and magazines, and after doing television commercials, she began taking acting lessons. Moynahan made her television debut in a guest appearance in the comedy series Sex and the City in 1999, where she later had a recurring role as Natasha.\nShe made her feature film debut in Coyote Ugly (2000). She was cast in a supporting role in Serendipity (2001). Moynahan was also featured in the action film The Sum of All Fears (2002), spy thriller The Recruit (2003), the science-fiction movie I, Robot (2004), and the political thriller Lord of War (2005).\nShe starred in the ABC television series Six Degrees, which premiered in September 2006, but was cancelled after one season. Moynahan completed work in Elizabeth Allen's Ramona

#

In [21]:
# auto reload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [212]:
# shuffle df with seed 111
df = df_grp.copy()

In [358]:
df_grp.sample(3)

,sentence_id,source,sentence,claims,source_text,sentence_context,source_sentence,source_ngram,source_sents
3622,4966,Despacito,"On March 17, 2017, both a solo pop version and a salsa version featuring Puerto Rican musician Victor Manuelle were released.","[Despacito has a version which features a Puerto Rican musician., Despacito has a version which features Victor Manuelle., Despacito had multiple versions as of March 17, 2017.]","Despacito\n""Despacito"" ([despa ` sito]; English: ""Slowly"") is a single by Puerto Rican pop artist Luis Fonsi featuring Puerto Rican rapper Daddy Yankee from the former's upcoming studio album. On January 13, 2017, Universal Music Latin released ""Despacito"" and its music video, which shows both artists performing the song in the La Perla neighborhood of Old San Juan, Puerto Rico and the local bar La Factoría. The song was written by Luis Fonsi, Erika Ender and Daddy Yankee, and was produced by Andrés Tor...","[Despacito, Several remix versions were released following the single's success., On March 17, 2017, both a solo pop version and a salsa version featuring Puerto Rican musician Victor Manuelle were released., On April 17, 2017, a remix version featuring Canadian singer Justin Bieber was released; this version helped to improve the song's chart performance in numerous countries, particularly in the English-speaking world.]","On March 17, 2017, both a solo pop version and a salsa version featuring Puerto Rican musician Victor Manuelle were released.","[Despacito, Several remix versions were released following the single's success. , On March 17, 2017, both a solo pop version and a salsa version featuring Puerto Rican musician Victor Manuelle were released. , On April 17, 2017, a remix version featuring Canadian singer Justin Bieber was released; this version helped to improve the song's chart performance in numerous countries, particularly in the English-speaking world. ]","[Despacito\n, ""Despacito"" ([despa ` sito]; English: ""Slowly"") is a single by Puerto Rican pop artist Luis Fonsi featuring Puerto Rican rapper Daddy Yankee from the former's upcoming studio album. , On January 13, 2017, Universal Music Latin released ""Despacito"" and its music video, which shows both artists performing the song in the La Perla neighborhood of Old San Juan, Puerto Rico and the local bar La Factoría. , The song was written by Luis Fonsi, Erika Ender and Daddy Yankee, and was produced by And..."
2508,3336,Bermuda,He claimed the islands for the Spanish Empire.,"[Bermuda is an island claimed for the Spanish Empire., Bermuda is part of one of the largest empires in history.]","Bermuda\nBermuda ([pronbərˈmjuːdə]) is a British Overseas Territory in the North Atlantic Ocean. It is approximately 1070 km east-southeast of Cape Hatteras, North Carolina; 1236 km south of Cape Sable Island, Nova Scotia; and 1578 km north of San Juan, Puerto Rico. The capital city is Hamilton. Bermuda is an associate member of Caribbean Community (CARICOM).\nThe first person known to have reached Bermuda was the Spanish sea captain Juan de Bermúdez in 1503, after whom the islands are named. He claimed...","[Bermuda, The first person known to have reached Bermuda was the Spanish sea captain Juan de Bermúdez in 1503, after whom the islands are named., He claimed the islands for the Spanish Empire., Bermúdez never landed on the islands, but made two visits to the archipelago, of which he created a recognisable map.]",He claimed the islands for the Spanish Empire.,"[Bermuda, The first person known to have reached Bermuda was the Spanish sea captain Juan de Bermúdez in 1503, after whom the islands are named. , He claimed the islands for the Spanish Empire. , Bermúdez never landed on the islands, but made two visits to the archipelago, of which he created a recognisable map. ]","[Bermuda\n, Bermuda ([pronbərˈmjuːdə]) is a British Overseas Territory in the North Atlantic Ocean. , It is approximately 1070 km east-southeast of Cape Hatteras,

In [359]:
#drop source_sentence
if "source_sentence" in df.columns:
    df.drop(columns=["source_sentence"], inplace=True)

In [351]:
# strip sentence column
df_grp["sentence"] = df_grp["sentence"].apply(lambda x: x.strip())
# strip each element in sentence_context
df_grp["sentence_context"] = df_grp["sentence_context"].apply(lambda x: [el.strip() for el in x])
# strip each element in claims
df_grp["claims"] = df_grp["claims"].apply(lambda x: [el.strip() for el in x])

/tmp/ipykernel_1240981/582158496.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_grp["sentence"] = df_grp["sentence"].apply(lambda x: x.strip())
/tmp/ipykernel_1240981/582158496.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_grp["sentence_context"] = df_grp["sentence_context"].apply(lambda x: [el.strip() for el in x])
/tmp/ipykernel_1240981/582158496.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

In [360]:
# save df_grp to jsonl
df_grp.to_json("/mnt/data/factcheck/claim_extraction/feversum/all.jsonl", orient="records", lines=True)

In [361]:
df = df_grp.copy()

In [362]:
# set aut of source column
used_sources = set(df["source"].unique())
#len of overlap with source_map keys
len(used_sources & set(source_map.keys()))

2659

In [363]:
len(used_sources), len(df)

(2683, 4419)

In [364]:
df.sample(3)

,sentence_id,source,sentence,claims,source_text,sentence_context,source_sentence,source_ngram,source_sents
4259,5971,Cyndi Lauper,Her charitable efforts were acknowledged in 2013 when the singer was invited as a special guest to attend US President Barack Obama's second-term inaugural.,"[Cyndi Lauper was invited as a special guest to Barack Obama's second-term inaugural., Cyndi Lauper is a singer.]","Cyndi Lauper\nCynthia Ann Stephanie ""Cyndi"" Lauper (born June 22, 1953) is an American singer, songwriter, actress and LGBTQ activist. Her career has spanned over 30 years. Her debut solo album She's So Unusual (1983) was the first debut female album to chart four top-five hits on the Billboard Hot 100 - ""Girls Just Want to Have Fun"", ""Time After Time"", ""She Bop"", and ""All Through the Night"" - and earned Lauper the Best New Artist award at the 27th Grammy Awards in 1985. Her success continued with the s...","[Cyndi Lauper, Lauper has been celebrated for her humanitarian work, particularly as an advocate for LGBT rights in the United States., Her charitable efforts were acknowledged in 2013 when the singer was invited as a special guest to attend US President Barack Obama's second-term inaugural.]",Her charitable efforts were acknowledged in 2013 when the singer was invited as a special guest to attend US President Barack Obama's second-term inaugural.,"[Cyndi Lauper, Lauper has been celebrated for her humanitarian work, particularly as an advocate for LGBT rights in the United States. , Her charitable efforts were acknowledged in 2013 when the singer was invited as a special guest to attend US President Barack Obama's second-term inaugural. ]","[Cyndi Lauper\n, Cynthia Ann Stephanie ""Cyndi"" Lauper (born June 22, 1953) is an American singer, songwriter, actress and LGBTQ activist. , Her career has spanned over 30 years. , Her debut solo album She's So Unusual (1983) was the first debut female album to chart four top-five hits on the Billboard Hot 100 - ""Girls Just Want to Have Fun"", ""Time After Time"", ""She Bop"", and ""All Through the Night"" - and earned Lauper the Best New Artist award at the 27th Grammy Awards in 1985. , Her success continued w..."
1613,2161,Syd Barrett,"Best known as a founder member of the band Pink Floyd, Barrett was the lead singer, guitarist and principal songwriter in its early years and is credited with naming the band.","[Syd Barrett was a singer., Syd Barrett was a guitarist for Pink Floyd., Syd Barrett was the principal songwriter of Pink Floyd., Syd Barrett was the lead singer of Pink Floyd., Syd Barrett was a member of a band., Syd Barrett played the guitar., Syd Barrett is credited with naming the band Pink Floyd., Syd Barrett was a songwriter., Syd Barrett is a founding member of the band Pink Floyd.]","Syd Barrett\nRoger Keith ""Syd"" Barrett (6 January 1946 - 7 July 2006) was a British musician, composer, singer-songwriter, and painter. Best known as a founder member of the band Pink Floyd, Barrett was the lead singer, guitarist and principal songwriter in its early years and is credited with naming the band. Barrett was excluded from Pink Floyd in April 1968 after David Gilmour took over as their new guitarist, and was briefly hospitalised amid speculation of mental illness.\nBarrett was musically act...","[Syd Barrett, Roger Keith ""Syd"" Barrett (6 January 1946 - 7 July 2006) was a British musician, composer, singer-songwriter, and painter., Best known as a founder member of the band Pink Floyd, Barrett was the lead singer, guitarist and principal songwriter in its early years and is credited with naming the band., Barrett was excluded from Pink Floyd in April 1968 after David Gilmour took over as their new guitarist, and was briefly hospitalised amid speculation of mental illness.]","Best known as a founder member of the band Pink Floyd, Barrett was the lead singer, guitarist and principal songwriter in its early years and is credited with naming the band.","[Syd Barrett, Rog

In [127]:
# cast claims to list
df["claims"] = df["claims"].apply(list)
# aggregate by source and merge claims, first source, first source_text and list of source_sentence
df_final = df.groupby("sentence_id").agg({"claims": "sum", "source_text": "first", "source_sentence": list}).reset_index()
df_final["claims"] = df_final["claims"].apply(lambda x: list(set(x)))
# prepend id column , ordinal
df_final["id"] = range(1, len(df_final)+1)
# shuffle by 111 seed
df_final = df_final.sample(frac=1, random_state=111)
#rename source sentence to source_sentences
df_final.rename(columns={"source_sentence": "source_sentences"}, inplace=True)
#move id to first column
cols = df_final.columns.tolist()
cols = cols[-1:] + cols[:-1]
df_final = df_final[cols]
df_final

,id,source,claims,source_text,source_sentences
1300,1301,José Ferrer,"[José Ferrer was an actor., José Ferrer played the part of Cyrano de Bergerac., José Ferrer won a Tony Award in 1947., José ...","José Ferrer\nJosé Vicente Ferrer de Otero y Cintrón (January 8, 1912 - January 26, 1992), known as José Ferrer, was an Amer...",[1539]
392,393,Britney Spears,"[Britney Spears has an album named Baby One More Time., Britney Spears made at least two studio albums., Britney Spears rele...","Britney Spears\nBritney Jean Spears (born December 2, 1981) is an American singer, dancer, and actress. Born in McComb, Miss...","[2002, 3729]"
1935,1936,Peloponnesian War,"[Sparta won the Peloponnesian War., Sparta was the principal enemy of Athens, between 431 and 404 BC.]",Peloponnesian War\nThe Peloponnesian War (431 - 404 BC) was an ancient Greek war fought by Athens and its empire against the...,[6283]
1291,1292,Joker (character),"[Bill Finger is acknowledged for writing the Joker., The Joker is a character., The credit for the Joker's creation is dispu...","Joker (character)\nThe Joker is a fictional supervillain created by Bill Finger, Bob Kane, and Jerry Robinson who first appe...",[2125]
532,533,Coca,"[Coca is grown as a cash crop., Coca is grown in Bolivia., Coca is grown in Colombia., Coca is a plant., Coca is grown in Ar...","Coca\nCoca is any of the four cultivated plants in the family Erythroxylaceae, native to western South America.\nThe plant i...",[5173]
...,...,...,...,...,...
118,119,Amanda Peet,"[Amanda Peet was in the movie the X- Files: I Want to Believe., Amanda Peet has appeared in a variety of films., Amanda Peet...","Amanda Peet\nAmanda Peet (born January 11, 1972) is an American actress and author who has appeared in film, stage, and tele...",[4831]
681,682,Doctor Who (series 10),"[Doctor Who (series 10) is the thirty-sixth season of the show., Doctor Who (series 10) is part of the rebooted program.]",Doctor Who (series 10)\nThe tenth series of the British science fiction television programme Doctor Who premiered on 15 Apri...,[5213]
86,87,Alec Baldwin,"[Alec Baldwin won two Emmy Awards for his work on 30 Rock., Alec Baldwin won seven Screen Actors Guild Awards for 30 Rock., ...","Alec Baldwin\nAlexander Rae ""Alec"" Baldwin III (born April 3, 1958) is an American actor, writer, producer, and comedian. A ...",[1084]
724,725,Eat Pray Love,"[Eat Pray Love is a romantic comedy-drama film., Eat Pray Love is a 2010 biographical film., Eat Pray Love stars Julia Rober...",Eat Pray Love\nEat Pray Love is a 2010 American biographical romantic comedy-drama film starring Julia Roberts as Elizabeth ...,[4598]


In [425]:
df_final = df
# reorder columns source	sentence_id	sentence	claims	source_text	sentence_context to put sentence after source_text
df_final = df_final[["source", "sentence_id", "claims", "source_text",  "sentence", "sentence_context"]]

In [372]:
len(validation)

1027

In [373]:
test_sources = pd.read_json("/mnt/data/factcheck/claim_extraction/feversum/backup/test.jsonl", lines=True)[["source"]]
validation_sources = pd.read_json("/mnt/data/factcheck/claim_extraction/feversum/backup/validation.jsonl", lines=True)[["source"]]
# inner join test_sources to df_final
test = test_sources.merge(df_final, on="source", how="inner")
validation = validation_sources.merge(df_final, on="source", how="inner")
# the rest goes here
train = df_final[~df_final["source"].isin(test["source"]) & ~df_final["source"].isin(validation_sources["source"])]


In [426]:
len(train), len(test), len(validation)
# originally (3672, 453, 460)

(3530, 444, 445)

In [375]:
g= df.groupby("source").agg({"sentence_id":list}).reset_index()
g

,source,sentence_id
0,12-hour clock,[1492]
1,14th Dalai Lama,[1785]
2,19 Kids and Counting,"[4197, 5792]"
3,1983–84 NBA season,[1319]
4,1991 NBA Playoffs,"[2992, 4632]"
...,...,...
2678,Zindagi Na Milegi Dobara,[3995]
2679,Zinedine Zidane,"[3090, 3221]"
2680,Zlatan Ibrahimović,"[1664, 5228]"
2681,Zooey Deschanel,[318]


In [128]:
import numpy as np
train, validation, test = np.split(df_final, [int(.8*len(df_final)), int(.9*len(df_final))])

In [376]:
train

,source,sentence_id,claims,source_text,sentence,sentence_context
0,Chris Hemsworth,2,"[Chris Hemsworth appeared in A Perfect Getaway., Chris Hemsworth appeared in Star Trek., Chris Hemsworth is known for his role on the Australian TV series Home and Away., Chris Hemsworth is known for playing Thor in The Marvel Cinematic Universe., Chris Hemsworth is known for playing Kim Hyde.]","Chris Hemsworth\nChris Hemsworth (born 11 August 1983) is an Australian actor. He is known for playing Kim Hyde in the Australian TV series Home and Away (2004-07) and Thor in the Marvel Cinematic Universe since 2011. Hemsworth has also appeared in the science fiction action film Star Trek (2009), the thriller adventure A Perfect Getaway (2009), the horror comedy The Cabin in the Woods (2012), the dark-fantasy action film Snow White and the Huntsman (2012), the war film Red Dawn (2012), and the biograph...","Hemsworth has also appeared in the science fiction action film Star Trek (2009), the thriller adventure A Perfect Getaway (2009), the horror comedy The Cabin in the Woods (2012), the dark-fantasy action film Snow White and the Huntsman (2012), the war film Red Dawn (2012), and the biographical sports drama film Rush (2013).","[Chris Hemsworth, He is known for playing Kim Hyde in the Australian TV series Home and Away (2004-07) and Thor in the Marvel Cinematic Universe since 2011., Hemsworth has also appeared in the science fiction action film Star Trek (2009), the thriller adventure A Perfect Getaway (2009), the horror comedy The Cabin in the Woods (2012), the dark-fantasy action film Snow White and the Huntsman (2012), the war film Red Dawn (2012), and the biographical sports drama film Rush (2013)., In 2015, he starred in ..."
2,Ireland,5,"[Ireland has a central plain., Ireland has relatively low-lying mountains., Ireland is a place on Earth., Ireland has several navigable rivers., Ireland is surrounded by water.]","Ireland\nIreland ([ˈaɪərlənd] Éire [ˈeːɾʲə]; Ulster-Scots: Airlann [- scoˈɑːrlən]) is an island in the North Atlantic. It is separated from Great Britain to its east by the North Channel, the Irish Sea, and St George's Channel. Ireland is the second-largest island of the British Isles, the third-largest in Europe, and the twentieth-largest on Earth.\nPolitically, Ireland is divided between the Republic of Ireland (officially named Ireland), which covers five-sixths of the island, and Northern Ireland, w...","The island's geography comprises relatively low-lying mountains surrounding a central plain, with several navigable rivers extending inland.","[Ireland, Just under 4.6 million live in the Republic of Ireland and just over 1.8 million live in Northern Ireland., The island's geography comprises relatively low-lying mountains surrounding a central plain, with several navigable rivers extending inland., The island has lush vegetation, a product of its mild but changeable climate which avoids extremes in temperature.]"
3,Carlos Santana,6,"[Carlos Santana was born in the forties., Carlos Santana is a musician., Carlos Santana was in an American Latin rock band., Carlos Santana is Mexican and American., Carlos Santana formed Santana in 1966.]","Carlos Santana\nCarlos Santana (born July 20, 1947) is a Mexican and American musician who first became famous in the late 1960s and early 1970s with his band, Santana, which pioneered a fusion of rock and Latin American music. The band's sound featured his melodic, blues-based guitar lines set against Latin and African rhythms featuring percussion instruments such as timbales and congas not generally heard in rock music. Santana continued to work in these forms over the following decades. He experience...","Carlos Santana (born July 20, 1947) is a Mexican and American musician who first became famous in the late 1960s and early 1970s with his band, Santana, which pioneered a fusion of rock and Latin American music.","[Carlos Santana, Carlos Santana (born July 20, 1947) is a Mexican and American

In [238]:
len(wiki)

5416535

In [377]:
# shuffle all dfs with seed 111
train, validation, test, df_final = train.sample(frac=1, random_state=111), validation.sample(frac=1, random_state=111), test.sample(frac=1, random_state=111), df_final.sample(frac=1, random_state=111)

In [378]:
validation.head(4)

,source,sentence_id,claims,source_text,sentence,sentence_context
145,Stevie Ray Vaughan,4368,"[Stevie Ray Vaughn played in a band with an electric blues guitarist., Stevie Ray Vaughn played in Marc Benno's band.]","Stevie Ray Vaughan\nStephen Ray Vaughan (October 3, 1954 - August 27, 1990) was an American musician, singer, songwriter, and record producer. In spite of a short-lived mainstream career spanning seven years, he is widely considered one of the most influential guitarists in the history of music, and one of the most important figures in the revival of blues in the 1980s. AllMusic describes him as ""a rocking powerhouse of a guitarist who gave blues a burst of momentum in the'80s, with influence still felt...","He played gigs with numerous bands, earning a spot in Marc Benno's band, the Nightcrawlers, and later with Denny Freeman in the Cobras, with whom he continued to work through late 1977.","[Stevie Ray Vaughan, In 1971 he dropped out of high school, and moved to Austin the following year., He played gigs with numerous bands, earning a spot in Marc Benno's band, the Nightcrawlers, and later with Denny Freeman in the Cobras, with whom he continued to work through late 1977., He then formed his own group, Triple Threat Revue, before renaming the band Double Trouble after hiring drummer Chris Layton and bassist Tommy Shannon.]"
63,Yung Rich Nation,4142,"[Yung Rich Nation had numerous producers., Yung Rich Nation featured Young Thug., Yung Rich Nation was produced by a Canadian., Yung Rich Nation was produced by Zaytoven., Yung Rich Nation is an album.]","Yung Rich Nation\nYung Rich Nation is the debut studio album by American hip hop group Migos. It was released on July 31, 2015, by Quality Control Music, 300 Entertainment and Atlantic Records. The album features guest appearances from Chris Brown and Young Thug, while the production was handled by Zaytoven, Honorable C.N.O.T.E. and Murda Beatz, among others.","The album features guest appearances from Chris Brown and Young Thug, while the production was handled by Zaytoven, Honorable C.N.O.T.E. and Murda Beatz, among others.","[Yung Rich Nation, It was released on July 31, 2015, by Quality Control Music, 300 Entertainment and Atlantic Records., The album features guest appearances from Chris Brown and Young Thug, while the production was handled by Zaytoven, Honorable C.N.O.T.E. and Murda Beatz, among others.]"
193,The Prestige (film),2983,"[The Prestige is a film., The Prestige was released in 2006., The Prestige is about stage magicians.]","The Prestige (film)\nThe Prestige is a 2006 British-American mystery thriller film directed by Christopher Nolan, from a screenplay adapted by Nolan and his brother Jonathan from Christopher Priest's 1995 World Fantasy Award-winning novel of the same name. Its story follows Robert Angier and Alfred Borden, rival stage magicians in London at the end of the 19th century. Obsessed with creating the best stage illusion, they engage in competitive one-upmanship with tragic results. The film stars Hugh Jackma...","Along with The Illusionist and Scoop, The Prestige was one of three films released in 2006 to explore the world of stage magicians.","[The Prestige (film), The film was released on October 20, 2006, receiving positive reviews and strong box office results, and received Academy Award nominations for Best Cinematography and Best Art Direction., Along with The Illusionist and Scoop, The Prestige was one of three films released in 2006 to explore the world of stage magicians.]"
207,Quinoa,4390,"[Quinoa was domesticated for human consumption., Quinoa originated in South America.]","Quinoa\nQuinoa ([ˈkiːnwɑː] or [kɪˈnoʊ.ə], from Quechua kinwa or kinuwa) is the common name for Chenopodium quinoa, a flowering plant in the amaranth family Amaranthaceae. It is a herbaceous annual plant grown as a grain crop primarily for its edible seeds. Because it is not a grass, it is a pseudocereal rather than a true cereal. Quinoa is closely rel

In [379]:
if "pipeline_text" in df_final.columns:
    df_final.drop(columns="pipeline_text", inplace=True)
# save in utf-8 unicode
df_final.to_json("/mnt/data/factcheck/claim_extraction/feversum/all.jsonl", lines=True, orient="records", force_ascii=False)

# now save train, validation and test
train.to_json("/mnt/data/factcheck/claim_extraction/feversum/train.jsonl", lines=True, orient="records", force_ascii=False)
validation.to_json("/mnt/data/factcheck/claim_extraction/feversum/validation.jsonl", lines=True, orient="records", force_ascii=False)
test.to_json("/mnt/data/factcheck/claim_extraction/feversum/test.jsonl", lines=True, orient="records", force_ascii=False)

In [380]:
# expand train, validation, test by claim column
from collections import Counter
ctr = Counter()
def click(x):
    # increment counter by 1
    ctr[x] += 1
    return ctr[x]

train2 = train.explode("claims")
validation2 = validation.explode("claims")
test2 = test.explode("claims")
# rename column id to source_id
train2.rename(columns={"claims":"claim"}, inplace=True)
validation2.rename(columns={"claims":"claim"}, inplace=True)
test2.rename(columns={"claims":"claim"}, inplace=True)

# add id as {source_id}_{claim order in list}
train2["id"] = train2['sentence_id'].apply(lambda x: x*1000 + click(x))
validation2["id"] = validation2['sentence_id'].apply(lambda x: x*1000 + click(x))
test2["id"] = test2['sentence_id'].apply(lambda x: x*1000 + click(x))

# shuffle by 111 seed
train2 = train2.sample(frac=1, random_state=111)
validation2 = validation2.sample(frac=1, random_state=111)
test2 = test2.sample(frac=1, random_state=111)

In [381]:
#  source	sentence_id	claim	source_text	sentence	sentence_context	id -> now move id up front
train2 = train2[["id", "source", "sentence_id", "claim", "source_text", "sentence", "sentence_context"]]
validation2 = validation2[["id", "source", "sentence_id", "claim", "source_text", "sentence", "sentence_context"]]
test2 = test2[["id", "source", "sentence_id", "claim", "source_text", "sentence", "sentence_context"]]
train2

,id,source,sentence_id,claim,source_text,sentence,sentence_context
1842,2466002,Napoleon,2466,Napoleon was a commander.,"Napoleon\nNapoleon Bonaparte ([nəˈpoʊliən_ˈboʊnəpɑrt] [napɔleɔ̃ bɔnapaʁt]; 15 August 1769 - 5 May 1821) was a French military and political leader who rose to prominence during the French Revolution and led several successful campaigns during the French Revolutionary Wars. As Napoleon I, he was Emperor of the French from 1804 until 1814, and again in 1815. Napoleon dominated European and global affairs for more than a decade while leading France against a series of coalitions in the Napoleonic Wars. He ...","One of the greatest commanders in history, his wars and campaigns are studied at military schools worldwide.","[Napoleon, He won most of these wars and the vast majority of his battles, building a large empire that ruled over continental Europe before its final collapse in 1815., One of the greatest commanders in history, his wars and campaigns are studied at military schools worldwide., Napoleon's political and cultural legacy has endured as one of the most celebrated and controversial leaders in human history.]"
370,481003,Sense and Sensibility,481,Sense and Sensibility went on sale in 1813.,"Sense and Sensibility\nSense and Sensibility is a novel by Jane Austen, published in 1811. It was published anonymously; By A Lady appears on the cover page where the author's name might have been. It tells the story of the Dashwood sisters, Elinor and Marianne, both of age to marry.\nThe novel follows the young women to their new home with their widowed mother, a meagre cottage on the property of a distant relative, where they experience love, romance and heartbreak. The novel is set in southwest Engla...","The novel sold out its first print run of 750 copies in the middle of 1813, marking a success for its author, who then had a second print run later that year.","[Sense and Sensibility, The novel is set in southwest England, London and Sussex between 1792 and 1797., The novel sold out its first print run of 750 copies in the middle of 1813, marking a success for its author, who then had a second print run later that year., The novel continued in publication throughout the 19th, 20th and 21st centuries.]"
2824,3788004,Wars of the Roses,3788,The Wars of the Roses happened in the 15th century.,"Wars of the Roses\nThe Wars of the Roses were a series of wars for control of the throne of England fought between supporters of two rival branches of the royal House of Plantagenet: the House of Lancaster (associated with a red rose), and the House of York (whose symbol was a white rose). The conflict lasted through many sporadic episodes between 1455 and 1487; however, there was fighting before and after this period between the houses. The power struggle ignited around social and financial troubles fo...","The House of Tudor ruled the Kingdom of England until 1603, with the death of Elizabeth I, granddaughter of Henry VII and Elizabeth of York.","[Wars of the Roses, After assuming the throne as Henry VII, he married Elizabeth of York, the eldest daughter and heir of Edward IV, thereby uniting the two claims., The House of Tudor ruled the Kingdom of England until 1603, with the death of Elizabeth I, granddaughter of Henry VII and Elizabeth of York.]"
2046,2727002,Taran Killam,2727,Taran Killam is a writer.,"Taran Killam\nTaran Hourie Killam (born April 1, 1982) is an American actor, comedian, and writer. He is best known for his television work on shows such as The Amanda Show, Wild'n Out, Mad TV, and Saturday Night Live. Killam is also known for his portrayal of a teen pop star in the Disney Channel Original Movie Stuck in the Suburbs.\nKillam recently performed the role of King George III in the Broadway production of Hamilton"" at the Richard Rodgers Theatre, ending his run on the evening of April 13, 20...","Taran Hourie Killam (born April 1, 1982) is an American actor, comedian, and writer.","[Taran Killam, Taran Hour

In [444]:
validation2

,id,source,sentence_id,claim,source_text,sentence,sentence_context
205,4956002,Blink-182,4956,Blink-182 helped develop the genre pop punk.,"Blink-182\nBlink-182 (often stylized as blink-182; pronounced ""blink one eighty two"") is an American rock band formed in Poway, California in 1992. The band currently consists of bassist and vocalist Mark Hoppus, drummer Travis Barker, and guitarist and vocalist Matt Skiba. Founded by guitarist and vocalist Tom DeLonge, bassist and vocalist Mark Hoppus and drummer Scott Raynor, the band emerged from the Southern California punk scene of the early 1990s and first gained notoriety for high-energy live sho...",Blink-182 is considered a key group in the development of pop punk; the band's combination of pop melodies with fast-paced punk rock featured a more radio-friendly accessibility than prior bands.,"[Blink-182, The band's seventh studio album, California, was released on July 1, 2016., Blink-182 is considered a key group in the development of pop punk; the band's combination of pop melodies with fast-paced punk rock featured a more radio-friendly accessibility than prior bands., The trio has sold over thirteen million albums in the United States, and over 50 million albums worldwide.]"
399,501001,Anushka Sharma,501,Anushka Sharma starred in Band Baaja Baaraat.,"Anushka Sharma\nAnushka Sharma ([əˈnʊʂkaː ˈʃərmaː]; born 1 May 1988) is an Indian film actress, producer and model. She has established a career in Hindi films, and is one of the most popular and highest-paid actresses in India. She is the recipient of numerous accolades, including one Filmfare Award from seven nominations.\nBorn in Ayodhya and raised in Bangalore, Sharma had her first modelling assignment for the fashion designer Wendell Rodricks in 2007 and later moved to Mumbai to pursue a full-time ...",Sharma rose to prominence with starring roles in Yash Raj Films's romances Band Baaja Baaraat (2010) and Jab Tak Hai Jaan (2012); the latter won her a Best Supporting Actress award at Filmfare.,"[Anushka Sharma, Following a successful audition at Yash Raj Films, she made her screen debut opposite Shah Rukh Khan in the blockbuster romance Rab Ne Bana Di Jodi (2008), which earned a Filmfare Award for Best Actress nomination., Sharma rose to prominence with starring roles in Yash Raj Films's romances Band Baaja Baaraat (2010) and Jab Tak Hai Jaan (2012); the latter won her a Best Supporting Actress award at Filmfare., She had her biggest commercial successes with the roles of a television reporter..."
402,6125003,Low (David Bowie album),6125,David Bowie has an album called Heroes.,"Low (David Bowie album)\nLow is the eleventh studio album by English musician David Bowie, released on RCA Records on 14 January 1977. Recorded following Bowie's move to West Berlin after a period of drug addiction and personal instability, Low became the first of three collaborations with musician Brian Eno and producer Tony Visconti, later termed the ""Berlin Trilogy"". The album was in fact recorded largely in France, and marked a shift in Bowie's musical style toward an electronic and avant-garde appr...","The album was in fact recorded largely in France, and marked a shift in Bowie's musical style toward an electronic and avant-garde approach that would be further explored on subsequent albums ""Heroes"" (1977) and Lodger (1979).","[Low (David Bowie album), Recorded following Bowie's move to West Berlin after a period of drug addiction and personal instability, Low became the first of three collaborations with musician Brian Eno and producer Tony Visconti, later termed the ""Berlin Trilogy""., The album was in fact recorded largely in France, and marked a shift in Bowie's musical style toward an electronic and avant-garde approach that would be further explored on subsequent albums ""Heroes"" (1977) and Lodger (1979)., Though it was i..."
234,2660001,Heidi Klum,2660,Heidi Klum made a cameo appearance in Perfect Stranger.,"Heidi Klum\nHeidi Klum ([

In [445]:
# in train2, validation2, test2 concat sentence_context by \n
train2["sentence_context"] = train2["sentence_context"].apply(lambda x: "\n".join(x))   
validation2["sentence_context"] = validation2["sentence_context"].apply(lambda x: "\n".join(x))
test2["sentence_context"] = test2["sentence_context"].apply(lambda x: "\n".join(x))

In [446]:
# save as jsonl to /mnt/data/factcheck/claim_extraction/feversum/hf
train2.to_json("/mnt/data/factcheck/claim_extraction/feversum/hf/train.jsonl", lines=True, orient="records")
validation2.to_json("/mnt/data/factcheck/claim_extraction/feversum/hf/validation.jsonl", lines=True, orient="records")
test2.to_json("/mnt/data/factcheck/claim_extraction/feversum/hf/test.jsonl", lines=True, orient="records")
import datasets
#save as arrow
train_ds = datasets.Dataset.from_pandas(train2)
validation_ds = datasets.Dataset.from_pandas(validation2)
test_ds = datasets.Dataset.from_pandas(test2)
# make a DatasetDict
ds = datasets.DatasetDict({"train": train_ds, "validation": validation_ds, "test": test_ds})
# save as arrow
ds.save_to_disk("/mnt/data/factcheck/claim_extraction/feversum/hf2")


Saving the dataset (0/1 shards):   0%|          | 0/13620 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1678 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1700 [00:00<?, ? examples/s]

In [48]:
# save df to parquet
df_grp.to_parquet("/mnt/data/factcheck/fever/2022_error_correction/intermediate_masks/train_dev_gold.parquet")

In [16]:
df_grp= pd.read_parquet("/mnt/data/factcheck/fever/2022_error_correction/intermediate_masks/train_dev_gold.parquet")

In [93]:
from openai import OpenAI
# Modify OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "EMPTY"
openai_api_base = "http://g02:8000/v1"
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)
completion = client.completions.create(model="mistralai/Mistral-7B-Instruct-v0.2",
                                      prompt="San Francisco is a")
print("Completion result:", completion)

Completion result: Completion(id='cmpl-631724c31a6d49118d224c5c232dfca9', choices=[CompletionChoice(finish_reason='length', index=0, logprobs=None, text=' must-see city for many travelers, but it’s also a very')], created=1022605, model='mistralai/Mistral-7B-Instruct-v0.2', object='text_completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=16, prompt_tokens=5, total_tokens=21))


In [272]:
%pip install tortus

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.0/347.0 kB 9.2 MB/s eta 0:00:000:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 48.4 MB/s eta 0:00:00m eta 0:00:010:0102
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.9/466.9 kB 30.3 MB/s eta 0:00:00
  Created wheel for jupyter-contrib-nbextensions: filename=jupyter_contrib_nbextensions-0.7.0-py2.py3-none-any.whl size=23428778 sha256=ee480129d283aeb2f718db7f0b16583c1c37eb9ab86a77e3a1b7cd319656f3be
  Stored in directory: /home/ullriher/.cache/pip/wheels/ea/cc/7d/99ef154f984726b1201c0f72cfe9c9d7c5132c1a2ae4d8677f
  Created wheel for jupyter_contrib_core: filename=jupyter_contrib_core-0.4.2-py2.py3-none-any.whl size=17482 sha256=ebe07346e07481861f23ece740bd9517bdda7ddd9864e49a2dbfafadc1a99999
  Stored in directory: /home/ullriher/.cache/pip/wheels/a9/52/88/e0643cdfd68f0562087918c37dd583378648dbc3df68b907f7
Successfully built jupyter-contrib

In [387]:
df_final.sample(3)

,source,sentence_id,claims,source_text,sentence,sentence_context
1345,Chester Bennington,1787,"[Chester Bennington is an American., Chester Bennington is an actor., Chester Bennington is a singer., Chester Bennington is a songwriter., Chester Bennington is a musician.]","Chester Bennington\nChester Charles Bennington (born March 20, 1976) is an American musician, singer, songwriter and actor. He is best known as the lead vocalist of rock bands Linkin Park, Dead by Sunrise, and live rock cover band Bucket of Weenies. He was originally the lead vocalist for Sean Dowdell And His Friends? and Grey Daze. Bennington was the lead vocalist for Stone Temple Pilots from 2013 to 2015.\nBennington became known as a vocalist with Linkin Park's debut album, Hybrid Theory, in 2000, wh...","Chester Charles Bennington (born March 20, 1976) is an American musician, singer, songwriter and actor.","[Chester Bennington, Chester Charles Bennington (born March 20, 1976) is an American musician, singer, songwriter and actor., He is best known as the lead vocalist of rock bands Linkin Park, Dead by Sunrise, and live rock cover band Bucket of Weenies.]"
1417,Maria I of Portugal,1884,"[Maria I of Portugal was born in 1734., Maria I of Portugal was born in December., Maria I of Portugal was Queen of a place derived from the words Gharb al-Andalus., Maria I of Portugal was Queen of Portugal., Maria I of Portugal was Queen of Brazil., Maria I of Portugal was a monarch in the Kingdom of Portugal., Maria I of Portugal did not live to see the deposition of the Portuguese monarchy., Maria I of Portugal was known in English as Mary I., Maria I of Portugal was also known as Dona Maria I., Mar...","Maria I of Portugal\nDona Maria I (English: Mary I; 17 December 1734 - 20 March 1816) was Queen of Portugal, Brazil, and the Algarves. Known as Maria the Pious (in Portugal), or Maria the Mad (in Brazil), she was the first undisputed Queen regnant of Portugal. With Napoleon's European conquests, her court, then under the direction of Prince Dom João, the Prince Regent, moved to Brazil, then a Portuguese colony. Later on, Brazil would be elevated from the rank of a colony to that of a kingdom, with the c...","Dona Maria I (English: Mary I; 17 December 1734 - 20 March 1816) was Queen of Portugal, Brazil, and the Algarves.","[Maria I of Portugal, Dona Maria I (English: Mary I; 17 December 1734 - 20 March 1816) was Queen of Portugal, Brazil, and the Algarves., Known as Maria the Pious (in Portugal), or Maria the Mad (in Brazil), she was the first undisputed Queen regnant of Portugal.]"
1305,CONCACAF Champions League,1733,"[The CONCACAF Champions League is organized for football clubs in Central America., The CONCACAF Champions League is an annual continental football club competition., The CONCACAF Champions League is organized for football clubs in North America., The CONCACAF Champions League is organized for football clubs in the Caribbean., The CONCACAF Champions League is organized for the top football clubs., The CONCACAF Champions League is organized by CONCACAF.]","CONCACAF Champions League\nThe CONCACAF Champions League is an annual continental football club competition organized by CONCACAF for the top football clubs in North America, Central America and the Caribbean. The winner of the CONCACAF Champions League qualifies for the FIFA Club World Cup. The tournament is officially known as the Scotiabank CONCACAF Champions League due to sponsorship reasons.\nThe tournament's current format uses a knockout format, though the tournament had a group stage prior to th...","The CONCACAF Champions League is an annual continental football club competition organized by CONCACAF for the top football clubs in North America, Central America and the Caribbean.","[CONCACAF Champions League, The CONCACAF Champions League is an annual continental football club competition organized by CONCACAF for the top football clubs in North America, Central America and the Caribbean., The wi

In [396]:
bullet = "\n  •"
df_final["annotable_text"] = df_final.apply(lambda x: f"{bullet.join(['CLAIMS']+x['claims'])}\n---\nSENTENCE\n{x['source']}: {x['sentence']}\n---\n{bullet.join(['CONTEXT']+x['sentence_context'])}\n---\nARTICLE\n{x['source_text']}", axis=1)

In [431]:
# get sentence_id -> annotation map from df_annotated
# df_annotated = pd.read_json("/mnt/data/factcheck/claim_extraction/feversum/annotated.jsonl", lines=True)
df_annotated2 = df_annotated[["sentence_id", "annotation"]]
#add annotations to df_final
df_final = df_final.merge(df_annotated2, on="sentence_id", how="left")

In [432]:
df_final.sample(3)

,source,sentence_id,claims,source_text,sentence,sentence_context,annotation
2993,Lexmark,4170,"[Lexmark manufactures laser printers and imaging products., Lexmark is owned by China., Lexmark is a corporation., Lexmark is based in the United States.]","Lexmark\nLexmark International, Inc. is a Chinese-owned corporation based in the United States that manufactures laser printers and imaging products. The company is headquartered in Lexington, Kentucky.","Lexmark International, Inc. is a Chinese-owned corporation based in the United States that manufactures laser printers and imaging products.","[Lexmark, Lexmark International, Inc. is a Chinese-owned corporation based in the United States that manufactures laser printers and imaging products., The company is headquartered in Lexington, Kentucky.]",None
3011,Xbox One,4191,"[The Xbox One was announced in May of 2013., The Xbox One is the successor to the Xbox 360., The Xbox One is the third console in a brand owned by Microsoft.]","Xbox One\nXbox One is a line of home video game consoles developed by Microsoft. Announced in May 2013, it is the successor to Xbox 360 and the third console in the Xbox family. It was first released in North America, parts of Europe, Australia, and Brazil in November 2013, and in Japan, China, and other European countries in September 2014. It is the first Xbox game console to be released in China, specifically in the Shanghai Free-Trade Zone. Microsoft marketed the device as an ""all-in-one entertainme...","Announced in May 2013, it is the successor to Xbox 360 and the third console in the Xbox family.","[Xbox One, Xbox One is a line of home video game consoles developed by Microsoft., Announced in May 2013, it is the successor to Xbox 360 and the third console in the Xbox family., It was first released in North America, parts of Europe, Australia, and Brazil in November 2013, and in Japan, China, and other European countries in September 2014.]",None
3004,Gray wolf,4182,"[Humans are a threat to the gray wolf., The Gray wolf is threatened by tigers., The Gray wolf is at the top of the food chain.]","Gray wolf\nThe gray wolf or grey wolf (Canis lupus), also known as the timber wolf or western wolf, is a canine native to the wilderness and remote areas of Eurasia and North America. It is the largest extant member of its family, with males averaging 43 - and females 36 -. Like the red wolf, it is distinguished from other Canis species by its larger size and less pointed features, particularly on the ears and muzzle. Its winter fur is long and bushy and predominantly a mottled gray in color, although n...","The gray wolf is typically an apex predator throughout its range, with only humans and tigers posing a serious threat to it.","[Gray wolf, It is a social animal, travelling in nuclear families consisting of a mated pair, accompanied by the pair's adult offspring., The gray wolf is typically an apex predator throughout its range, with only humans and tigers posing a serious threat to it., It feeds primarily on large ungulates, though it also eats smaller animals, livestock, carrion, and garbage.]",None


In [435]:
annotated_ids = df_final[df_final["annotation"].notna()]["sentence_id"].tolist()
#shuffle annotated_ids
annotated_ids = np.random.permutation(annotated_ids)
# show first 5 annotated sentences
df_final[df_final["sentence_id"].isin(annotated_ids[:3])]

,source,sentence_id,claims,source_text,sentence,sentence_context,annotation
1192,Marion Cotillard,1642,"[Marion Cotillard portrayed a singer., Marion Cotillard acted in La Vie En Rose (2007)., Marion Cotillard played a French cabaret singer., Marion Cotillard won a Golden Globe Award., Marion Cotillard achieved worldwide recognition.]","Marion Cotillard\nMarion Cotillard ([ma.ʁjɔ̃ kɔ.ti.jaʁ]; born 30 September 1975) is a French actress, singer-songwriter, musician, environmentalist and spokesperson for Greenpeace who achieved international fame with the film La Vie en Rose (2007). She is the recipient of an Academy Award, a BAFTA Award, a Golden Globe Award, two César Awards, a European Award, and a Lumières Award. Cotillard has appeared in more than 50 feature films and is also known for being the face of Lady Dior handbags since 2008...","For her portrayal of the French singer Édith Piaf in the biopic La Vie en Rose (2007), Cotillard achieved worldwide recognition and won her second César Award, a BAFTA Award, a Golden Globe Award, a Lumières Award and the Academy Award for Best Actress, becoming the first and (as of 2017) only actor to win an Academy Award for a performance in French, and also the second actress and one of only six actors to have won this award for a part spoken in a foreign language.","[Marion Cotillard, Since 2003, Cotillard has alternated between French and English-language roles in mainstream and art-house productions, and has received international acclaim and awards for her performances in films such as Tina Lombardi in A Very Long Engagement (2004), for which she won her first César Award for Best Supporting Actress., For her portrayal of the French singer Édith Piaf in the biopic La Vie en Rose (2007), Cotillard achieved worldwide recognition and won her second César Award, a B...",1
2384,The New York Times,3278,"[The New York Times is published by The New York Times Company., The New York Times is a daily newspaper., The New York Times has been continuously published since 1851., The New York Times is American., The New York Times was founded in New York City.]","The New York Times\nThe New York Times (sometimes abbreviated NYT and The Times) is an American daily newspaper, founded and continuously published in New York City since September 18, 1851, by The New York Times Company. The New York Times has won 122 Pulitzer Prizes, more than any other newspaper.\nThe paper's print version in 2013 had the second-largest circulation, behind The Wall Street Journal, and the largest circulation among the metropolitan newspapers in the United States. The New York Times i...","The New York Times (sometimes abbreviated NYT and The Times) is an American daily newspaper, founded and continuously published in New York City since September 18, 1851, by The New York Times Company.","[The New York Times, The New York Times (sometimes abbreviated NYT and The Times) is an American daily newspaper, founded and continuously published in New York City since September 18, 1851, by The New York Times Company., The New York Times has won 122 Pulitzer Prizes, more than any other newspaper.]",1
4386,Ron Cobb,6472,"[Ron Cobb is a film director., Ron Cobb was born in 1937.]","Ron Cobb\nRon Cobb (born 1937) is an American cartoonist, artist, writer, film designer, and film director.\nBy the age of 18, with no formal training in graphic illustration, Cobb was working as an animation ""inbetweener"" artist for Disney Studios in Burbank, California. He progressed to becoming a breakdown artist on the animation feature Sleeping Beauty (1959). (This was the last Disney film to have cels inked by hand.) After Sleeping Beauty was completed in 1957, Disney laid off Cobb and he spent ...","Ron Cobb (born 1937) is an American cartoonist, artist, writer, film designer, and film director.","[Ron Cobb, Ron Cobb (born 1937) is an American cartoonist, artist, writer, film designer, and film director., By the age of 18, with no formal training

In [440]:
len(df_final)

4419

In [436]:
df_annotated[df_annotated["sentence_id"].isin(annotated_ids[:3])]

,source,sentence_id,claims,source_text,sentence,sentence_context,annotable_text,annotation
2469,The New York Times,3278,"[The New York Times is published by The New York Times Company., The New York Times is a daily newspaper., The New York Times has been continuously published since 1851., The New York Times is American., The New York Times was founded in New York City.]","The New York Times\nThe New York Times (sometimes abbreviated NYT and The Times) is an American daily newspaper, founded and continuously published in New York City since September 18, 1851, by The New York Times Company. The New York Times has won 122 Pulitzer Prizes, more than any other newspaper.\nThe paper's print version in 2013 had the second-largest circulation, behind The Wall Street Journal, and the largest circulation among the metropolitan newspapers in the United States. The New York Times i...","The New York Times (sometimes abbreviated NYT and The Times) is an American daily newspaper, founded and continuously published in New York City since September 18, 1851, by The New York Times Company.","[The New York Times, The New York Times (sometimes abbreviated NYT and The Times) is an American daily newspaper, founded and continuously published in New York City since September 18, 1851, by The New York Times Company., The New York Times has won 122 Pulitzer Prizes, more than any other newspaper.]","CLAIMS\n •The New York Times is published by The New York Times Company.\n •The New York Times is a daily newspaper.\n •The New York Times has been continuously published since 1851.\n •The New York Times is American.\n •The New York Times was founded in New York City.\n---\nSENTENCE\nThe New York Times: The New York Times (sometimes abbreviated NYT and The Times) is an American daily newspaper, founded and continuously published in New York City since September 18, 1851, by The New York Times Comp...",1
4548,Ron Cobb,6472,"[Ron Cobb is a film director., Ron Cobb was born in 1937.]","Ron Cobb\nRon Cobb (born 1937) is an American cartoonist, artist, writer, film designer, and film director.\nBy the age of 18, with no formal training in graphic illustration, Cobb was working as an animation ""inbetweener"" artist for Disney Studios in Burbank, California. He progressed to becoming a breakdown artist on the animation feature Sleeping Beauty (1959). (This was the last Disney film to have cels inked by hand.) After Sleeping Beauty was completed in 1957, Disney laid off Cobb and he spent ...","Ron Cobb (born 1937) is an American cartoonist, artist, writer, film designer, and film director.","[Ron Cobb, Ron Cobb (born 1937) is an American cartoonist, artist, writer, film designer, and film director., By the age of 18, with no formal training in graphic illustration, Cobb was working as an animation ""inbetweener"" artist for Disney Studios in Burbank, California.]","CLAIMS\n •Ron Cobb is a film director.\n •Ron Cobb was born in 1937.\n---\nSENTENCE\nRon Cobb: Ron Cobb (born 1937) is an American cartoonist, artist, writer, film designer, and film director.\n---\nCONTEXT\n •Ron Cobb\n •Ron Cobb (born 1937) is an American cartoonist, artist, writer, film designer, and film director.\n •By the age of 18, with no formal training in graphic illustration, Cobb was working as an animation ""inbetweener"" artist for Disney Studios in Burbank, California.\n---\nARTICLE\nR...",1
1235,Marion Cotillard,1642,"[Marion Cotillard portrayed a singer., Marion Cotillard acted in La Vie En Rose (2007)., Marion Cotillard played a French cabaret singer., Marion Cotillard won a Golden Globe Award., Marion Cotillard achieved worldwide recognition.]","Marion Cotillard\nMarion Cotillard ([ma.ʁjɔ̃ kɔ.ti.jaʁ]; born 30 September 1975) is a French actress, singer-songwriter, musician, environmentalist and spokesperson for Greenpeace who achieved international fame with the film La Vie en Rose (2007). She is the recipient of an Academy Award, a BAFTA Award, a Golden Globe Award, two César Aw

In [286]:
print(df_final["annotable_text"][0])

CLAIMS
  •Chris Hemsworth appeared in A Perfect Getaway.
  •Chris Hemsworth appeared in Star Trek.
  •Chris Hemsworth is known for his role on the Australian TV series Home and Away.
  •Chris Hemsworth is known for playing Thor in The Marvel Cinematic Universe.
  •Chris Hemsworth is known for playing Kim Hyde.
---
SENTENCE
Chris Hemsworth: Hemsworth has also appeared in the science fiction action film Star Trek (2009), the thriller adventure A Perfect Getaway (2009), the horror comedy The Cabin in the Woods (2012), the dark-fantasy action film Snow White and the Huntsman (2012), the war film Red Dawn (2012), and the biographical sports drama film Rush (2013).
---
CONTEXT
  •Chris Hemsworth
  •He is known for playing Kim Hyde in the Australian TV series Home and Away (2004-07) and Thor in the Marvel Cinematic Universe since 2011.
  •Hemsworth has also appeared in the science fiction action film Star Trek (2009), the thriller adventure A Perfect Getaway (2009), the horror comedy The Cabi

In [441]:
len(df_final)

4419

In [301]:
df_final["annotation"] = None
# df_final = df_final.sample(frac=1, random_state=111)

In [427]:
#add column for annotation
from IPython.display import display, clear_output
# shuffle
# randomly print unannotated rows, prompt user for 1-4
for i, row in df_final.iterrows():
    if row["annotation"] is None:
        clear_output()
        print("Progress: ", len(df_final[df_final["annotation"].notna()]) , " => ",len(df_final[df_final["annotation"].notna()])/len(df_final))
        print(row["annotable_text"], flush=True)
        annotation = input("1-5: 1=perfect, 2=all claims covered but not perfect, 3=most claims covered, 4=some,5=none of the claims covered:")
        if annotation in "12345":
            df_final.at[i, "annotation"] = annotation
        else:
            print("Invalid input")
        break

KeyError: 'annotation'

In [417]:
# print a ratio of annotated rows
len(df_final[df_final["annotation"].notna()])/len(df_final)

0.02014030323602625

In [419]:
annotated = df_final[df_final["annotation"].notna()]
# summed len of claims column values
annotated["claims"].apply(len).sum()

350

In [420]:
len(annotated)

89

In [423]:
len(df_final)

4419

In [421]:
# show percentages of each value
df_final["annotation"].value_counts(normalize=True)

1    0.921348
4    0.033708
2    0.022472
3    0.011236
5    0.011236
Name: annotation, dtype: float64

In [422]:
0.921348+0.022472

0.94382

In [336]:
df_annotated=df_final.copy()

In [334]:
# show rows where 4 or 5
df_final[df_final["annotation"].isin(["4","5"])]

,source,sentence_id,claims,source_text,sentence,sentence_context,annotable_text,annotation
506,Physical Graffiti,669,"[Led Zeppelin signed a deal with Atlantic Records., Led Zeppelin put out the album Physical Graffiti in 1975., Led Zeppelin put out the album Led Zeppelin II in 1969., Led Zeppelin put out the album Led Zeppelin in 1969., Led Zeppelin was first named the New Yardbirds.]","Physical Graffiti\nPhysical Graffiti is the sixth studio album by the English rock band Led Zeppelin, released as a double album on 24 February 1975 by their newly founded imprint label Swan Song Records. The band wrote and recorded eight new songs for the album at Headley Grange, which stretched the total time of the record beyond the typical length of a single LP, so the band decided to make Physical Graffiti a double album by including unreleased tracks from earlier recording sessions: one outtake fr...","Physical Graffiti is the sixth studio album by the English rock band Led Zeppelin, released as a double album on 24 February 1975 by their newly founded imprint label Swan Song Records.","[Physical Graffiti, Physical Graffiti is the sixth studio album by the English rock band Led Zeppelin, released as a double album on 24 February 1975 by their newly founded imprint label Swan Song Records., The band wrote and recorded eight new songs for the album at Headley Grange, which stretched the total time of the record beyond the typical length of a single LP, so the band decided to make Physical Graffiti a double album by including unreleased tracks from earlier recording sessions: one outtake ...","CLAIMS\n •Led Zeppelin signed a deal with Atlantic Records.\n •Led Zeppelin put out the album Physical Graffiti in 1975.\n •Led Zeppelin put out the album Led Zeppelin II in 1969.\n •Led Zeppelin put out the album Led Zeppelin in 1969.\n •Led Zeppelin was first named the New Yardbirds.\n---\nSENTENCE\nPhysical Graffiti: Physical Graffiti is the sixth studio album by the English rock band Led Zeppelin, released as a double album on 24 February 1975 by their newly founded imprint label Swan Song Reco...",4
3784,Dangal (film),5218,"[Aparshakti Khurana appeared in Dangal., Aparshakti Khurana is an actor., Aparshakti Khurana works in Bollywood.]","Dangal (film)\nDangal (Wrestling competition) is a 2016 Indian Hindi-language biographical sports drama film directed by Nitesh Tiwari. Produced by Walt Disney Pictures, Aamir Khan Productions and UTV Motion Pictures, it stars Aamir Khan as Mahavir Singh Phogat, an amateur wrestler, who trains his daughters Geeta Phogat and Babita Kumari to be world-class wrestlers, both of who go on to win medals at the 2010 Commonwealth Games. Fatima Sana Shaikh and Sanya Malhotra play the older selves of the sisters ...","Fatima Sana Shaikh and Sanya Malhotra play the older selves of the sisters while Sakshi Tanwar play their mother and Aparshakti Khurana, their cousin.","[Dangal (film), Produced by Walt Disney Pictures, Aamir Khan Productions and UTV Motion Pictures, it stars Aamir Khan as Mahavir Singh Phogat, an amateur wrestler, who trains his daughters Geeta Phogat and Babita Kumari to be world-class wrestlers, both of who go on to win medals at the 2010 Commonwealth Games., Fatima Sana Shaikh and Sanya Malhotra play the older selves of the sisters while Sakshi Tanwar play their mother and Aparshakti Khurana, their cousin., The development of the film began in early...","CLAIMS\n •Aparshakti Khurana appeared in Dangal.\n •Aparshakti Khurana is an actor.\n •Aparshakti Khurana works in Bollywood.\n---\nSENTENCE\nDangal (film): Fatima Sana Shaikh and Sanya Malhotra play the older selves of the sisters while Sakshi Tanwar play their mother and Aparshakti Khurana, their cousin.\n---\nCONTEXT\n •Dangal (film)\n •Produced by Walt Disney Pictures, Aamir Khan Productions and UTV Motion Pictures, it stars Aamir Khan as Mahavir Singh Phogat, an amateur wrestler, who trains hi...",4
4494,Legion (2010 film),6372,"[Legion stars Dan Ste

In [ ]:
# count rows where annota

In [328]:
#see where annotation is an empty string
df_final[df_final["annotation"] == ""]
# set it to "1"
df_final.at[3102, "annotation"] = "1"


,source,sentence_id,claims,source_text,sentence,sentence_context,annotable_text,annotation


In [442]:
# save annotated df to jsonl
df_final.to_json("/mnt/data/factcheck/claim_extraction/feversum/annotated.jsonl", orient="records", lines=True)

In [424]:
len(train2)+len(validation2)+len(test2) 

16998

In [429]:
len(df_final)

4419

In [430]:
df_final.sample(3)

,source,sentence_id,claims,source_text,sentence,sentence_context
3390,Dr. Dre,4613,"[Dr. Dre was previously the co-owner of Death Row Records., Dr. Dre was previously an artist on Death Row Records., Dr. Dre co-owned Death Row Records., Dr. Dre was an artist on Death Row Records.]","Dr. Dre\nAndre Romelle Young (born February 18, 1965), better known by his stage name Dr. Dre, is an American rapper, record producer, and entrepreneur. He is the founder and current CEO of Aftermath Entertainment and Beats Electronics. Dre was previously the co-owner of, and an artist on, Death Row Records. He has produced albums for and overseen the careers of many rappers, including 2Pac, The D.O.C., Snoop Dogg, Eminem, Xzibit, Knoc-turn ` al, 50 Cent, The Game and Kendrick Lamar. He is credited as a...","Dre was previously the co-owner of, and an artist on, Death Row Records.","[Dr. Dre, He is the founder and current CEO of Aftermath Entertainment and Beats Electronics., Dre was previously the co-owner of, and an artist on, Death Row Records., He has produced albums for and overseen the careers of many rappers, including 2Pac, The D.O.C., Snoop Dogg, Eminem, Xzibit, Knoc-turn ` al, 50 Cent, The Game and Kendrick Lamar.]"
169,One World Trade Center,212,"[One World Trade Center's structure was completed on August 30, 2012., One World Trade Center has a steel structure.]","One World Trade Center\nOne World Trade Center (also known as 1 World Trade Center, 1 WTC or Freedom Tower) is the main building of the rebuilt World Trade Center complex in Lower Manhattan, New York City. It is the tallest building in the Western Hemisphere, and the sixth-tallest in the world. The supertall structure has the same name as the North Tower of the original World Trade Center, which was completely destroyed in the terrorist attacks of September 11, 2001. The new skyscraper stands on the nor...","The tower's steel structure was topped out on August 30, 2012.","[One World Trade Center, One World Trade Center became the tallest structure in New York City on April 30, 2012, when it surpassed the height of the Empire State Building., The tower's steel structure was topped out on August 30, 2012., On May 10, 2013, the final component of the skyscraper's spire was installed, making the building, including its spire, reach a total height of 1776 ft.]"
3453,Kangana Ranaut,4698,"[Kangana Ranaut trained under an Indian theatre director., Kangana Ranaut's feature film debut was in an Indian romantic crime film., Kangana Ranaut was awarded the Filmfare Award for Best Female Debut., Kangana Ranaut's feature film debut was in 2006.]","Kangana Ranaut\nKangana Ranaut ([kəŋɡənaː raːɳoːʈʰ]; born 23 March 1986) is an Indian film actress. She has established a career in Bollywood and is one of the highest-paid actresses in India. Ranaut is particularly known in the media for expressing her honest opinions in public and is frequently credited as one of the most fashionable Indian celebrities. She is the recipient of several awards, including three National Film Awards and Filmfare Awards in four categories.\nBorn in Bhambla, a small town in...","After training under the theatre director Arvind Gaur, Ranaut made her feature film debut in the 2006 thriller Gangster, for which she was awarded the Filmfare Award for Best Female Debut.","[Kangana Ranaut, Determined to build her own career path, she relocated to Delhi at age sixteen, where she briefly became a model., After training under the theatre director Arvind Gaur, Ranaut made her feature film debut in the 2006 thriller Gangster, for which she was awarded the Filmfare Award for Best Female Debut., She received praise for portraying emotionally intense characters in the dramas Woh Lamhe (2006), Life in a...]"
